In [4]:
# !pip install pysqlite3

In [1]:
import sqlite3
from sqlite3 import Error

import pandas as pd
import os
from pathlib import Path
import numpy as np
import re
import openpyxl
import datetime
from datetime import datetime as dt
from datetime import date, timedelta
from pandas import ExcelWriter
from openpyxl import load_workbook

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
try:
    sqlite_connection = sqlite3.connect('third_stepik.db')
    cursor = sqlite_connection.cursor()
    print("База данных создана и успешно подключена к SQLite")

    sqlite_select_query = "select sqlite_version();"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Версия базы данных SQLite: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")

База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.39.4',)]
Соединение с SQLite закрыто


In [5]:
try:
    sqlite_connection = sqlite3.connect('second_stepik.db')
    sqlite_create_table_query = '''
                CREATE TABLE Location(id SERIAL, value TEXT);
                CREATE TABLE Conference(id SERIAL, value TEXT);
                CREATE TABLE Paper(id SERIAL, title TEXT, conference TEXT, location TEXT);

                INSERT INTO conference(id,  value) VALUES (1, 'CONF14' );
                INSERT INTO conference(id,  value) VALUES (2, 'CONF13' );
                INSERT INTO conference(id,  value) VALUES (3, 'CONF12' );
                INSERT INTO conference(id,  value) VALUES (4, 'CONF11' );


                INSERT INTO "location" (id,  value) VALUES (1, 'spb' );
                INSERT INTO "location" (id,  value) VALUES (2, 'msc' );
                INSERT INTO "location" (id,  value) VALUES (3, 'vlg' );
                INSERT INTO "location" (id,  value) VALUES (4, 'kzn' );
                
                INSERT INTO paper VALUES (1, 'title1', 'CONF14', 'spb');
                INSERT INTO paper VALUES (2, 'title2', 'CONF14', 'vlg');
                INSERT INTO paper VALUES (3, 'title3', 'CONF13', 'spb');
                INSERT INTO paper VALUES (4, 'title4', 'NOT_EXISTED', 'spb');
    '''

    cursor = sqlite_connection.cursor()
    print("База данных подключена к SQLite")
    cursor.executescript(sqlite_create_table_query)
    sqlite_connection.commit()
except Exception as e: print(e)

База данных подключена к SQLite


In [7]:
try:
    sqlite_connection = sqlite3.connect('second_stepik.db')
    sqlite_create_table_query = '''
                SELECT * from paper
    '''

    cursor = sqlite_connection.cursor()
    print("query:\n")
    test_sql_df = cursor.executescript(sqlite_create_table_query)
    # sqlite_connection.commit()
except Exception as e: print(e)

query:



In [12]:
paper_df = pd.read_sql_query("SELECT * FROM paper", sqlite_connection)
conference_df = pd.read_sql_query("SELECT * FROM conference", sqlite_connection)
location_df = pd.read_sql_query("SELECT * FROM location", sqlite_connection)

In [88]:
# paper_df

In [89]:
# conference_df

In [90]:
# location_df

In [ ]:
SELECT DISTINCT поле FROM имя_таблицы WHERE условие

In [87]:
for row in cursor.execute('''SELECT conference FROM paper
                                WHERE conference NOT IN
                                (SELECT value FROM conference)
                                UNION
                                SELECT conference FROM paper
                                GROUP BY conference
                                HAVING COUNT(DISTINCT location)>1'''):
    print(row)

('CONF14',)
('NOT_EXISTED',)


In [3]:
try:
    sqlite_connection = sqlite3.connect('third_stepik.db')
    sqlite_create_table_query = '''
CREATE TABLE Spacecraft(
  id SERIAL PRIMARY KEY,
  name TEXT UNIQUE,
  service_life INT DEFAULT 1000,
  birth_year INT CHECK(birth_year > 0)
);

CREATE TABLE Planet(
  id SERIAL PRIMARY KEY,
  name TEXT UNIQUE,
  distance NUMERIC(5,2)
);

CREATE TABLE Commander(
  id SERIAL PRIMARY KEY,
  name TEXT
);

CREATE TABLE Flight(
  id INT PRIMARY KEY,
  spacecraft_id INT REFERENCES Spacecraft,
  planet_id INT REFERENCES Planet,
  commander_id INT REFERENCES Commander,
  start_date DATE,
  UNIQUE(spacecraft_id, start_date),
  UNIQUE(commander_id, start_date)
);
'''

    cursor = sqlite_connection.cursor()
    print("База данных подключена к SQLite")
    cursor.executescript(sqlite_create_table_query)
    sqlite_connection.commit()
except Exception as e: print(e)



База данных подключена к SQLite


In [12]:
# !pip install psycopg2-binary
# encoding: utf-8
# import psycopg2 as pg_driver

In [11]:
# Postgres, запущенный в докере, не требует пароля, если клиент находится на localhost
%time
# db = pg_driver.connect(user="postgres", host="localhost")
sqlite_connection = sqlite3.connect('third_stepik.db')
# cur = db.cursor()
cur = sqlite_connection.cursor()

def fun1(commander_id):
    result = []
    cur.execute("SELECT commander_id, spacecraft_id, start_date FROM Flight")
    for flight in cur.fetchall():
        if flight[0] != commander_id:
            continue
        cur.execute("SELECT id, name FROM Spacecraft")
    for spacecraft in cur.fetchall():
        if spacecraft[0] == flight[1]:
            result.append((spacecraft, flight)) # добавляем в результат пару
    return result

def fun2(commander_id):
    result = []
    for spacecraft, flight in fun1(commander_id):
        cur.execute("SELECT id, name FROM Commander")
        for cmndr in cur.fetchall():
            if cmndr[0] == flight[0]:
                result.append((cmndr[1], spacecraft[1], flight[2])) # добавляем в результат тройку
    return result

for row in fun2(1):
    print(row)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [ ]:
SELECT Commander.name, Spacecraft.name, start_date FROM Flight
LEFT JOIN Spacecraft ON Flight.spacecraft_id=Spacecraft.id
LEFT JOIN Commander ON Flight.commander_id=Commander.id
WHERE Commander.id=6

In [14]:
5 * 300100

1500500

In [5]:
path_to_test_xlsx = Path(os.getcwd()).parent.absolute()
path_to_test_xlsx = str(path_to_test_xlsx) + '\\'

In [77]:
cols_name = pd.read_excel(f'{path_to_test_xlsx}Наименования_столбцов_для_сборки_всех_приемов_онколога.xlsx', sheet_name = 0)
cols_name = cols_name.fillna('—')
cols_name_dict = {}
for _ in cols_name.columns.tolist():
    cols_name_dict[_] = [i for i in cols_name[_].values.tolist() if i != '—']
    
# читаем таблицу с крааткими намиенованиями ЦАОПов для ПРИЕМОВ ОНКОЛОГА
true_caop = pd.read_excel(f'{path_to_test_xlsx}СЛОВАРЬ_ЦАОП_2.xlsx')

In [78]:
print(f'Начало сборки: {dt.now()}')
start_reading = dt.now()


oncologist_consultations_visits_list = ['Первичная консультация +',
                                       'Повторная консультация +',
                                       'Повторная после ОК +',
                                       'Справки и направления +'
                                      ]

confirmed_diagnosis_visits_list = ['КО в рамках ДН +',
                                   'Выписка рецепта на ЛП +',
                                   'Снятие с учета +'
                                  ]

common_survey_visits_list = ['Общий осмотр']



oncologist_consultations_all_visits_dfs_list = []
confirmed_diagnosis_all_visits_dfs_list = []
common_survey_all_visits_dfs_list = []


# добавить столбец с датой выгрузки для проверки дублей

for _ in os.listdir(path_to_test_xlsx):
    if '.xlsx' in str(_):
        print(f'Читение: {_}')
        for w_sheet in load_workbook(path_to_test_xlsx + _, read_only=True).sheetnames:

            if w_sheet in oncologist_consultations_visits_list:
                print(w_sheet)
                target_df = pd.read_excel(path_to_test_xlsx + _,
                                    sheet_name = w_sheet,
                                    skiprows = 1,
                                    dtype=str)
                target_df['тип приема'] = w_sheet
    #             ######### дописать ИМЯ ФАЙЛА
                cols_quantity = target_df.shape[1]
                print(cols_quantity)
                try:
                    target_df.columns = cols_name_dict[cols_quantity]
                except:
                    print(f'Количество столбцов на листе {w_sheet} отличается от стандартного - необходимо изучить ошибку!')
                oncologist_consultations_all_visits_dfs_list.append(target_df)

            elif w_sheet in confirmed_diagnosis_visits_list:
                print(w_sheet)
                target_df = pd.read_excel(path_to_test_xlsx + _,
                                    sheet_name = w_sheet,
                                    skiprows = 1,
                                    dtype=str)
                target_df['тип приема'] = w_sheet
                cols_quantity = target_df.shape[1]
                print(cols_quantity)
                try:
                    target_df.columns = cols_name_dict[cols_quantity]
                except:
                    print(f'Количество столбцов на листе {w_sheet} отличается от стандартного - необходимо изучить ошибку!')
                confirmed_diagnosis_all_visits_dfs_list.append(target_df)

            elif w_sheet in common_survey_visits_list:
                print(w_sheet)
                target_df = pd.read_excel(path_to_test_xlsx + _,
                                    sheet_name = w_sheet,
                                    skiprows = 1,
                                    dtype=str)
                cols_quantity = target_df.shape[1]
                print(cols_quantity)
                try:
                    target_df.columns = cols_name_dict[cols_quantity]
                except:
                    print(f'Количество столбцов на листе {w_sheet} отличается от стандартного - необходимо изучить ошибку!')
                common_survey_all_visits_dfs_list.append(target_df)

            else: print(f'Такого наименования листов раньше не встречалось - {w_sheet}, что делать?')
    
all_oncologist_consultation = pd.concat(oncologist_consultations_all_visits_dfs_list, axis=0)
all_oncologist_consultation = all_oncologist_consultation.reset_index(drop = True)

all_confirmed_diagnosis_visits = pd.concat(confirmed_diagnosis_all_visits_dfs_list, axis=0)
all_confirmed_diagnosis_visits = all_confirmed_diagnosis_visits.reset_index(drop = True)

all_common_surveys = pd.concat(common_survey_all_visits_dfs_list, axis=0)
all_common_surveys = all_common_surveys.reset_index(drop = True)


end_reading = dt.now()

print(f'Приемы: первичный, повторный, после ОК, выписка справок - размер таблицы: {all_oncologist_consultation.shape}',
      f'Приемы: в рамках ДН, для направления на лечение, снятие с учета - размер таблицы: {all_confirmed_diagnosis_visits.shape}',
      f'Общий осмтор - размер таблицы: {all_common_surveys.shape}',
      f'\nНачало сборки:{start_reading}',
      f'Конец сборки: {end_reading}',
      sep='\n')

Начало сборки: 2022-05-18 12:49:43.954447
Читение: onko_new_20220514.xlsx
Первичная консультация +
80
Повторная консультация +
210
КО в рамках ДН +
102
Повторная после ОК +
75
Справки и направления +
29
Выписка рецепта на ЛП +
62
Снятие с учета +
22
Общий осмотр
94
Читение: Наименования_столбцов_для_сборки_всех_приемов_онколога.xlsx
Такого наименования листов раньше не встречалось - Sheet1 (3), что делать?
Такого наименования листов раньше не встречалось - кол-ов признаков по типам Прием, что делать?
Такого наименования листов раньше не встречалось - проверка корректности значений, что делать?
Читение: СЛОВАРЬ_ЦАОП_2.xlsx
Такого наименования листов раньше не встречалось - Модель, что делать?
Такого наименования листов раньше не встречалось - Лист1, что делать?
Приемы: первичный, повторный, после ОК, выписка справок - размер таблицы: (979, 262)
Приемы: в рамках ДН, для направления на лечение, снятие с учета - размер таблицы: (305, 143)
Общий осмтор - размер таблицы: (1284, 94)

Начало с

In [79]:
all_oncologist_consultation.shape, all_confirmed_diagnosis_visits.shape, all_common_surveys.shape

((979, 262), (305, 143), (1284, 94))

In [80]:
%%time
# 1. сборка всех видов приемов кроме данных об общем осмотре

pre_total_data = pd.concat([all_oncologist_consultation, all_confirmed_diagnosis_visits])

total_data = pre_total_data.copy()

# удобный мне порядок столбцов
total_data = total_data[[
     'тип приема',
'общая информация - ID протокола',
 'общая информация - ID пациента',
 'общая информация - Дата и время подписания протокола',
 'общая информация - ФИОпациента',
 'общая информация - ДР пациента',
 'общая информация - ОМС пациента',
 'общая информация - ФИО врача',
 'общая информация - Должность врача',
 'общая информация - МО приема',
 'общая информация - Структурное подразделение',
 'общая информация - Филиал',
 'общая информация - Отделение',
 'общая информация - Специализация врача',
 'Прикрепление - ID участка прикрепления в МУ приема',
 'Прикрепление - Участок прикрепления в МУ приема',
 'Прикрепление - Прикрепления в других МО',
 'Анамнез',
 'Жалобы (предъявляет/не предъявляет)',
 'Жалобы',
 'Описание жалоб',
 'Считает себя больным с ',
 'Общее состояние',
 'Уровень сознания',
 'Нервно-психический статус',
 'НПС иное',
 'Состояние пациента (ECOG)',
 'Температура тела',
 'Локальный статус',
 'Направительный диагноз - МКБ-10',
 'Направительный диагноз - Код мкб-10',
 'Диагноз - МКБ-10 расшифровка',
 'Диагноз - Код МКБ-10',
    
 'Диагноз - Развернутый клинический диагноз',
 'Диагноз - Дата устанвоки диагноза',
 'Диагноз - Статус диагноза',
 'Диагноз - Первично-множественная опухоль (да/нет)',
 'Диагноз - Метод подтверждения диагноза',
 'Диагноз - Клиническая группа',
     'Диагноз - Основание диагноза',
    
    'Прочие рекомендации',

        
 'Новый диагноз - МКБ-10',
 'Новый диагноз - Код МКБ-10',
 'Новый диагноз - Дата установления нового диагноза',
 'Новый диагноз - Статус нового диагноза',
 'Единичная опухоль - Сторона поражения',
 'Диагноз - Морфологический тип опухоли',
 'Единичная опухоль - Остоятельства выявления опухоли',
 'Единичная опухоль - Стадия TNM',

 'Единичная опухоль - Классификация остаточной опухоли (R)',
 'Единичная опухоль - Гистопатологическая степень злокачественности (G)',
 'Единичная опухоль - Инвазия в лимфатические сосуды (L)',
 'Единичная опухоль - Венозная инвазия (V)',
 'Единичная опухоль - Периневральная инвазия (Pn)',
 'Единичная опухоль - Локализация отдаленных метастазов',
 'Единичная опухоль - Клиническая группа',
 'Единичная опухоль - Стадия опухолевого процесса',
    
'Биопсия - Требуется взятие биопсии?',
 'Биопсия - Причина, по которой взятие биопсии не требуется',
 'Биопсия - Дата проведения биопсии',
 'Биопсия - Отсутствие информации о дате проведения биопсии',
 'Биопсия - Где проведена биопсия',
 'Биопсия - Где будет проводиться биопсия (тип МО)',
 'Биопсия - Где будет проводиться биопсия (МО)',
 'Биопсия - Отсутствие информации где проведена биопсия',
        
 'Онкоконсилиум - Требуется проведение онкоконсилиума? (true)',
 'Онкоконсилиум - Плановая дата проведения онкоконсилиума',
 'Онкоконсилиум - Место проведения онкоконсилиума',

 'Специализированное лечение - Требуется направление на специализированное лечение',
 'Специализированное лечение - Хирургическое лечение',
 'Специализированное лечение - Место проведения хирургического лечения',
 'Специализированное лечение - Лучевое лечение',
 'Специализированное лечение - Лекарственное лечение',
 'Специализированное лечение - Где будет проводиться лекарственное лечение?',
 'Специализированное лечение - Место проведения лечения',
 'Специализированное лечение - Плановая дата проведения лекарственного лечения',
 'Специализированное лечение - Требуется назначение лекарственной терапии?',
 'Специализированное лечение - МНН/ТН препарата',
 'Специализированное лечение - Дозировка',
    
 'Документы на приеме Справки и направления - Тип документа',
 'Документы на приеме Справки и направления - Инструментальные обследования',
 'Документы на приеме Справки и направления - Лабораторные исследования',
 'Документы на приеме Справки и направления - Консультация специалиста',
 'Документы на приеме Справки и направления - Госпитализация',
 'Документы на приеме Справки и направления - Цель выдачи',

 'Вложенный диагноз на первичном приеме (1) - МКБ-10',
 'Вложенный диагноз на первичном приеме (1) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (1) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (1) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (1) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (1) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (1) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (2) - МКБ-10',
 'Вложенный диагноз на первичном приеме (2) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (2) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (2) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (2) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (2) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (2) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (3) - МКБ-10',
 'Вложенный диагноз на первичном приеме (3) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (3) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (3) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (3) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (3) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (3) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (4) - МКБ-10',
 'Вложенный диагноз на первичном приеме (4) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (4) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (4) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (4) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (4) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (4) - Статус диагноза',
 
 'Единичная опухоль - cT',
 'Единичная опухоль - cN',
 'Единичная опухоль - cM',
 'Единичная опухоль - pT',
 'Единичная опухоль - pN',
 'Единичная опухоль - pM',
    
    'Множественная опухоль(1) - МКБ-10',
 'Множественная опухоль(1) - Код МКБ-10',
 'Множественная опухоль(1) - Развернутый клинический диагноз',
 'Множественная опухоль(1) - Статус диагноза',
 'Множественная опухоль(1) - Дата установления диагноза',
 'Множественная опухоль(1) - Основание диагноза',
 'Множественная опухоль(1) - Метод подтверждения диагноза',
 'Множественная опухоль(1) - Признак основной опухоли',
 'Множественная опухоль(1) - Сторона поражения',
 'Множественная опухоль(1) - Морфологический тип опухоли',
 'Множественная опухоль(1) - Обстоятельства выявления опухоли',
 'Множественная опухоль(1) - Стадия TNM',
 'Множественная опухоль(1) - pT',
 'Множественная опухоль(1) - pN',
 'Множественная опухоль(1) - pM',
 'Множественная опухоль(1) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(1) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(1) - Периневральная инвазия (Pn)',
 'Множественная опухоль(1) - Венозная инвазия (V)',
 'Множественная опухоль(1) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(1) - Локализация отдаленных метастазов',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.1',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.2',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.3',
 'Множественная опухоль(1) - cT',
 'Множественная опухоль(1) - cN',
 'Множественная опухоль(1) - cM',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.4',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.5',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.6',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.7',
 'Множественная опухоль(1) - Стадия опухолевого процесса',
 'Множественная опухоль(1) - Клиническая группа',
    
 'Множественная опухоль(2) - МКБ-10',
 'Множественная опухоль(2) - Код МКБ-10',
 'Множественная опухоль(2) - Развернутый клинический диагноз',
 'Множественная опухоль(2) - Статус диагноза',
 'Множественная опухоль(2) - Дата установления диагноза',
 'Множественная опухоль(2) - Основание диагноза',
 'Множественная опухоль(2) - Метод подтверждения диагноза',
 'Множественная опухоль(2) - Признак основной опухоли',
 'Множественная опухоль(2) - Сторона поражения',
 'Множественная опухоль(2) - Морфологический тип опухоли',
 'Множественная опухоль(2) - Обстоятельства выявления опухоли',
 'Множественная опухоль(2) - Стадия TNM',
 'Множественная опухоль(2) - pT',
 'Множественная опухоль(2) - pN',
 'Множественная опухоль(2) - pM',
 'Множественная опухоль(2) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(2) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(2) - Периневральная инвазия (Pn)',
 'Множественная опухоль(2) - Венозная инвазия (V)',
 'Множественная опухоль(2) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.8',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.9',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.10',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.11',
 'Множественная опухоль(2) - cT',
 'Множественная опухоль(2) - cN',
 'Множественная опухоль(2) - cM',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.12',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.13',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.14',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.15',
 'Множественная опухоль(2) - Стадия опухолевого процесса',
 'Множественная опухоль(2) - Клиническая группа',
 'Множественная опухоль(3) - МКБ-10',
 'Множественная опухоль(3) - Код МКБ-10',
 'Множественная опухоль(3) - Развернутый клинический диагноз',
 'Множественная опухоль(3) - Статус диагноза',
 'Множественная опухоль(3) - Дата установления диагноза',
 'Множественная опухоль(3) - Основание диагноза',
 'Множественная опухоль(3) - Метод подтверждения диагноза',
 'Множественная опухоль(3) - Признак основной опухоли',
 'Множественная опухоль(3) - Сторона поражения',
 'Множественная опухоль(3) - Морфологический тип опухоли',
 'Множественная опухоль(3) - Обстоятельства выявления опухоли',
 'Множественная опухоль(3) - Стадия TNM',
 'Множественная опухоль(3) - pT',
 'Множественная опухоль(3) - pN',
 'Множественная опухоль(3) - pM',
 'Множественная опухоль(3) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(3) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(3) - Периневральная инвазия (Pn)',
 'Множественная опухоль(3) - Венозная инвазия (V)',
 'Множественная опухоль(3) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.16',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.17',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.18',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.19',
 'Множественная опухоль(3) - cT',
 'Множественная опухоль(3) - cN',
 'Множественная опухоль(3) - cM',
     'Множественная опухоль(3) - Локализация отдаленных метастазов.20',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.21',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.22',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.23',
 'Множественная опухоль(3) - Стадия опухолевого процесса',
 'Множественная опухоль(3) - Клиническая группа',
 'Множественная опухоль(4) - МКБ-10',
 'Множественная опухоль(4) - Код МКБ-10',
 'Множественная опухоль(4) - Развернутый клинический диагноз',
 'Множественная опухоль(4) - Статус диагноза',
 'Множественная опухоль(4) - Дата установления диагноза',
 'Множественная опухоль(4) - Основание диагноза',
 'Множественная опухоль(4) - Метод подтверждения диагноза',
 'Множественная опухоль(4) - Признак основной опухоли',
 'Множественная опухоль(4) - Сторона поражения',
 'Множественная опухоль(4) - Морфологический тип опухоли',
 'Множественная опухоль(4) - Обстоятельства выявления опухоли',
 'Множественная опухоль(4) - Стадия TNM',
 'Множественная опухоль(4) - pT',
 'Множественная опухоль(4) - pN',
 'Множественная опухоль(4) - pM',
 'Множественная опухоль(4) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(4) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(4) - Периневральная инвазия (Pn)',
 'Множественная опухоль(4) - Венозная инвазия (V)',
 'Множественная опухоль(4) - Инвазия в лимфатические сосуды (L)',
     'Множественная опухоль(4) - Локализация отдаленных метастазов.24',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.25',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.26',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.27',
 'Множественная опухоль(4) - cT',
 'Множественная опухоль(4) - cN',
 'Множественная опухоль(4) - cM',
     'Множественная опухоль(4) - Локализация отдаленных метастазов.28',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.29',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.30',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.31',
 'Множественная опухоль(4) - Стадия опухолевого процесса',
 'Множественная опухоль(4) - Клиническая группа'
]]

Wall time: 18 ms


In [81]:
print('-=< Start >=-')
start_func_wt = dt.now()


# функция которая форматирует данные выгрузки (в том виде в котором мы получаем ее сейчас

# на входе принимает total_data - таблица, которую необходимо обработать
# предобработка описана ниже

############################################################################################



######### НЕОБХОДИМА ПЕРИОДИЧЕСКАЯ ПРОВЕРКА состояния форматов в выгрузке #########
# = '2020-01-15'
first_date_type_list = ['Биопсия - Дата проведения биопсии',
                        'Специализированное лечение - Плановая дата проведения лекарственного лечения']

# = '1997-04-05 00:00:00'
dob = ['общая информация - ДР пациента']

# = '1997-04-05T00:00:00+04:00'
second_date_type_list = ['общая информация - Дата и время подписания протокола',
                         'Считает себя больным с ',
                        'Диагноз - Дата устанвоки диагноза',
                        'Новый диагноз - Дата установления нового диагноза',
                        'Онкоконсилиум - Плановая дата проведения онкоконсилиума',                         
                        'Вложенный диагноз на первичном приеме (1) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (2) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (3) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (4) - Дата установления диагноза',
                        'Множественная опухоль(1) - Дата установления диагноза', 
                        # понять почему остался постфикс в конце
                        'Множественная опухоль(2) - Дата установления диагноза',
                        'Множественная опухоль(3) - Дата установления диагноза',
                        'Множественная опухоль(4) - Дата установления диагноза']

date_cols = first_date_type_list+dob+second_date_type_list

############################################################################################

# цикл обработки дат к формату dt.date

for _ in total_data.columns.tolist():
    if _ in date_cols:
        total_data[_] = total_data[_].fillna(pd.NaT)
        if _ == 'общая информация - ДР пациента':
            total_data[_] = pd.to_datetime(total_data[_], format = '%Y-%m-%d %H:%M:%S', errors='ignore').dt.date
        
        elif _ in second_date_type_list:
            try:
                total_data[_] = total_data[_].str.replace(r'T.+','')
            except:
                print(f'error here:\n{_}')
            
            total_data[_] = pd.to_datetime(total_data[_], format = '%Y-%m-%d', errors='ignore').dt.date
        elif _ in total_data:
            total_data[_] = pd.to_datetime(total_data[_], format = '%Y-%m-%d', errors='ignore').dt.date

total_data = total_data.sort_values(by='общая информация - Дата и время подписания протокола',
                                        ascending=False).copy()
print(f'даты успешно форматированы и отсортированы в порядке убывания Даты приема ("Дата и время подписания протокола")\n')
# 
print(f'подгружает значения ЦАОП...........')
total_data['ЦАОП'] = total_data['общая информация - МО приема'].map(true_caop.set_index('Названия строк')['ЦАОП'])

group_patients_caop_by_id = total_data.groupby('общая информация - ID пациента')
group_patients_caop_by_id = group_patients_caop_by_id.apply(lambda x: x['ЦАОП'].unique())
group_patients_caop_by_id = group_patients_caop_by_id.to_frame()
group_patients_caop_by_id['общая информация - ID пациента'] = group_patients_caop_by_id.index
group_patients_caop_by_id.columns = ['ЦАОП_id', 'общая информация - ID пациента']

group_second = total_data.groupby('общая информация - ОМС пациента')
group_patients_caop_by_oms = group_second.apply(lambda x: x['ЦАОП'].unique())
group_patients_caop_by_oms = group_patients_caop_by_oms.to_frame()
group_patients_caop_by_oms['общая информация - ОМС пациента'] = group_patients_caop_by_oms.index
group_patients_caop_by_oms.columns = ['ЦАОП_ОМС', 'общая информация - ОМС пациента']




total_data_caop = total_data.merge(group_patients_caop_by_id.rename_axis("index"),
                                                                        how='left',
                                                                        left_on='общая информация - ID пациента',
                                                                        right_on='общая информация - ID пациента')

total_data_caop = total_data_caop.merge(group_patients_caop_by_oms.rename_axis("index"),
                                                                        how='left',
                                                                        left_on='общая информация - ОМС пациента',
                                                                        right_on='общая информация - ОМС пациента')



# варварская распаковка всех ЦАОП пациента, если их было несколько за обрабатываемый период

for _ in ['ЦАОП_id', 'ЦАОП_ОМС']:
    total_data_caop[_] = total_data_caop[_].astype(str)
    total_data_caop[_] = total_data_caop[_].str.replace("['","", regex=False)
    total_data_caop[_] = total_data_caop[_].str.replace("']","", regex=False)
    total_data_caop[_] = total_data_caop[_].str.replace("' '",", 2) ", regex=False)


# сравнивает где длиннее строка в ячейке и забирает более полные данные

total_data_caop['ЦАОП_в'] = np.where(len(total_data_caop['ЦАОП_id']) < len(total_data_caop['ЦАОП_ОМС']),
                                                                            total_data_caop['ЦАОП_ОМС'],
                                                                            total_data_caop['ЦАОП_id'])        


print('Кол-во строк по значениям для столбца ЦАОП:',
      f"{total_data_caop['ЦАОП_в'].value_counts()}",
     '\n\nЕсли встречаются неподгруженные значения это нужно разобрать сравнив спрочник с исходником выгрузки!',
     sep='\n')

end_func_wt = dt.now()            
print(f'start_func_work_time: {start_func_wt}\nend_func_work_time: {end_func_wt}')

-=< Start >=-
даты успешно форматированы и отсортированы в порядке убывания Даты приема ("Дата и время подписания протокола")

подгружает значения ЦАОП...........


<ipython-input-81-f2d9dec47a52>:52: FutureWarning: The default value of regex will change from True to False in a future version.
  total_data[_] = total_data[_].str.replace(r'T.+','')


Кол-во строк по значениям для столбца ЦАОП:
ММКЦ Коммунарка ЦАОП      406
ГКОБ 1                    317
МКНЦ им. А.С. Логинова    164
ГКБ им. С.П. Боткина      135
ГКБ им. Д.Д. Плетнёва     131
МГОБ 62                   113
Морозовская ДГКБ           12
ГКБ 52                      6
Name: ЦАОП_в, dtype: int64


Если встречаются неподгруженные значения это нужно разобрать сравнив спрочник с исходником выгрузки!
start_func_work_time: 2022-05-18 12:49:46.267231
end_func_work_time: 2022-05-18 12:49:46.672885


In [97]:
total_data_caop.columns.tolist()

['тип приема',
 'общая информация - ID протокола',
 'общая информация - ID пациента',
 'общая информация - Дата и время подписания протокола',
 'общая информация - ФИОпациента',
 'общая информация - ДР пациента',
 'общая информация - ОМС пациента',
 'общая информация - ФИО врача',
 'общая информация - Должность врача',
 'общая информация - МО приема',
 'общая информация - Структурное подразделение',
 'общая информация - Филиал',
 'общая информация - Отделение',
 'общая информация - Специализация врача',
 'Прикрепление - ID участка прикрепления в МУ приема',
 'Прикрепление - Участок прикрепления в МУ приема',
 'Прикрепление - Прикрепления в других МО',
 'Анамнез',
 'Жалобы (предъявляет/не предъявляет)',
 'Жалобы',
 'Описание жалоб',
 'Считает себя больным с ',
 'Общее состояние',
 'Уровень сознания',
 'Нервно-психический статус',
 'НПС иное',
 'Состояние пациента (ECOG)',
 'Температура тела',
 'Локальный статус',
 'Направительный диагноз - МКБ-10',
 'Направительный диагноз - Код мкб-10'

In [ ]:
gluer_cols = ['тип приема',
 'общая информация - ID протокола',
 'общая информация - ID пациента',
 'общая информация - Дата и время подписания протокола',
 'общая информация - ФИОпациента',
 'общая информация - ДР пациента',
 'общая информация - ОМС пациента',
 'общая информация - ФИО врача',
 'общая информация - Должность врача',
 'общая информация - МО приема',
 'общая информация - Структурное подразделение',
 'общая информация - Филиал',
 'общая информация - Отделение',
 'общая информация - Специализация врача',
 'Прикрепление - ID участка прикрепления в МУ приема',
 'Прикрепление - Участок прикрепления в МУ приема',
 'Прикрепление - Прикрепления в других МО',
 'Анамнез',
 'Жалобы (предъявляет/не предъявляет)',
 'Жалобы',
 'Описание жалоб',
 'Считает себя больным с ',
 'Общее состояние',
 'Уровень сознания',
 'Нервно-психический статус',
 'НПС иное',
 'Состояние пациента (ECOG)',
 'Температура тела',
 'Локальный статус',
 'Направительный диагноз - МКБ-10',
 'Направительный диагноз - Код мкб-10',
 'Диагноз - МКБ-10 расшифровка',
 'Диагноз - Код МКБ-10',
 'Диагноз - Развернутый клинический диагноз',
 'Диагноз - Дата устанвоки диагноза',
 'Диагноз - Статус диагноза',
 'Диагноз - Первично-множественная опухоль (да/нет)',
 'Диагноз - Метод подтверждения диагноза',
 'Диагноз - Клиническая группа',
 'Диагноз - Основание диагноза',
 'Прочие рекомендации',
 'Новый диагноз - МКБ-10',
 'Новый диагноз - Код МКБ-10',
 'Новый диагноз - Дата установления нового диагноза',
 'Новый диагноз - Статус нового диагноза',
 'Единичная опухоль - Сторона поражения',
 'Диагноз - Морфологический тип опухоли',
 'Единичная опухоль - Остоятельства выявления опухоли',
 'Единичная опухоль - Стадия TNM',
 'Единичная опухоль - Классификация остаточной опухоли (R)',
 'Единичная опухоль - Гистопатологическая степень злокачественности (G)',
 'Единичная опухоль - Инвазия в лимфатические сосуды (L)',
 'Единичная опухоль - Венозная инвазия (V)',
 'Единичная опухоль - Периневральная инвазия (Pn)',
 'Единичная опухоль - Локализация отдаленных метастазов',
 'Единичная опухоль - Клиническая группа',
 'Единичная опухоль - Стадия опухолевого процесса',
 'Биопсия - Требуется взятие биопсии?',
 'Биопсия - Причина, по которой взятие биопсии не требуется',
 'Биопсия - Дата проведения биопсии',
 'Биопсия - Отсутствие информации о дате проведения биопсии',
 'Биопсия - Где проведена биопсия',
 'Биопсия - Где будет проводиться биопсия (тип МО)',
 'Биопсия - Где будет проводиться биопсия (МО)',
 'Биопсия - Отсутствие информации где проведена биопсия',
 'Онкоконсилиум - Требуется проведение онкоконсилиума? (true)',
 'Онкоконсилиум - Плановая дата проведения онкоконсилиума',
 'Онкоконсилиум - Место проведения онкоконсилиума',
 'Специализированное лечение - Требуется направление на специализированное лечение',
 'Специализированное лечение - Хирургическое лечение',
 'Специализированное лечение - Место проведения хирургического лечения',
 'Специализированное лечение - Лучевое лечение',
 'Специализированное лечение - Лекарственное лечение',
 'Специализированное лечение - Где будет проводиться лекарственное лечение?',
 'Специализированное лечение - Место проведения лечения',
 'Специализированное лечение - Плановая дата проведения лекарственного лечения',
 'Специализированное лечение - Требуется назначение лекарственной терапии?',
 'Специализированное лечение - МНН/ТН препарата',
 'Специализированное лечение - Дозировка',
 'Документы на приеме Справки и направления - Тип документа',
 'Документы на приеме Справки и направления - Инструментальные обследования',
 'Документы на приеме Справки и направления - Лабораторные исследования',
 'Документы на приеме Справки и направления - Консультация специалиста',
 'Документы на приеме Справки и направления - Госпитализация',
 'Документы на приеме Справки и направления - Цель выдачи',
 'Вложенный диагноз на первичном приеме (1) - МКБ-10',
 'Вложенный диагноз на первичном приеме (1) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (1) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (1) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (1) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (1) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (1) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (2) - МКБ-10',
 'Вложенный диагноз на первичном приеме (2) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (2) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (2) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (2) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (2) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (2) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (3) - МКБ-10',
 'Вложенный диагноз на первичном приеме (3) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (3) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (3) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (3) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (3) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (3) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (4) - МКБ-10',
 'Вложенный диагноз на первичном приеме (4) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (4) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (4) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (4) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (4) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (4) - Статус диагноза',
 'Единичная опухоль - cT',
 'Единичная опухоль - cN',
 'Единичная опухоль - cM',
 'Единичная опухоль - pT',
 'Единичная опухоль - pN',
 'Единичная опухоль - pM',
 'Множественная опухоль(1) - МКБ-10',
 'Множественная опухоль(1) - Код МКБ-10',
 'Множественная опухоль(1) - Развернутый клинический диагноз',
 'Множественная опухоль(1) - Статус диагноза',
 'Множественная опухоль(1) - Дата установления диагноза',
 'Множественная опухоль(1) - Основание диагноза',
 'Множественная опухоль(1) - Метод подтверждения диагноза',
 'Множественная опухоль(1) - Признак основной опухоли',
 'Множественная опухоль(1) - Сторона поражения',
 'Множественная опухоль(1) - Морфологический тип опухоли',
 'Множественная опухоль(1) - Обстоятельства выявления опухоли',
 'Множественная опухоль(1) - Стадия TNM',
 'Множественная опухоль(1) - pT',
 'Множественная опухоль(1) - pN',
 'Множественная опухоль(1) - pM',
 'Множественная опухоль(1) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(1) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(1) - Периневральная инвазия (Pn)',
 'Множественная опухоль(1) - Венозная инвазия (V)',
 'Множественная опухоль(1) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(1) - Локализация отдаленных метастазов',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.1',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.2',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.3',
 'Множественная опухоль(1) - cT',
 'Множественная опухоль(1) - cN',
 'Множественная опухоль(1) - cM',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.4',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.5',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.6',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.7',
 'Множественная опухоль(1) - Стадия опухолевого процесса',
 'Множественная опухоль(1) - Клиническая группа',
 'Множественная опухоль(2) - МКБ-10',
 'Множественная опухоль(2) - Код МКБ-10',
 'Множественная опухоль(2) - Развернутый клинический диагноз',
 'Множественная опухоль(2) - Статус диагноза',
 'Множественная опухоль(2) - Дата установления диагноза',
 'Множественная опухоль(2) - Основание диагноза',
 'Множественная опухоль(2) - Метод подтверждения диагноза',
 'Множественная опухоль(2) - Признак основной опухоли',
 'Множественная опухоль(2) - Сторона поражения',
 'Множественная опухоль(2) - Морфологический тип опухоли',
 'Множественная опухоль(2) - Обстоятельства выявления опухоли',
 'Множественная опухоль(2) - Стадия TNM',
 'Множественная опухоль(2) - pT',
 'Множественная опухоль(2) - pN',
 'Множественная опухоль(2) - pM',
 'Множественная опухоль(2) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(2) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(2) - Периневральная инвазия (Pn)',
 'Множественная опухоль(2) - Венозная инвазия (V)',
 'Множественная опухоль(2) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.8',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.9',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.10',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.11',
 'Множественная опухоль(2) - cT',
 'Множественная опухоль(2) - cN',
 'Множественная опухоль(2) - cM',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.12',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.13',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.14',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.15',
 'Множественная опухоль(2) - Стадия опухолевого процесса',
 'Множественная опухоль(2) - Клиническая группа',
 'Множественная опухоль(3) - МКБ-10',
 'Множественная опухоль(3) - Код МКБ-10',
 'Множественная опухоль(3) - Развернутый клинический диагноз',
 'Множественная опухоль(3) - Статус диагноза',
 'Множественная опухоль(3) - Дата установления диагноза',
 'Множественная опухоль(3) - Основание диагноза',
 'Множественная опухоль(3) - Метод подтверждения диагноза',
 'Множественная опухоль(3) - Признак основной опухоли',
 'Множественная опухоль(3) - Сторона поражения',
 'Множественная опухоль(3) - Морфологический тип опухоли',
 'Множественная опухоль(3) - Обстоятельства выявления опухоли',
 'Множественная опухоль(3) - Стадия TNM',
 'Множественная опухоль(3) - pT',
 'Множественная опухоль(3) - pN',
 'Множественная опухоль(3) - pM',
 'Множественная опухоль(3) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(3) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(3) - Периневральная инвазия (Pn)',
 'Множественная опухоль(3) - Венозная инвазия (V)',
 'Множественная опухоль(3) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.16',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.17',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.18',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.19',
 'Множественная опухоль(3) - cT',
 'Множественная опухоль(3) - cN',
 'Множественная опухоль(3) - cM',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.20',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.21',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.22',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.23',
 'Множественная опухоль(3) - Стадия опухолевого процесса',
 'Множественная опухоль(3) - Клиническая группа',
 'Множественная опухоль(4) - МКБ-10',
 'Множественная опухоль(4) - Код МКБ-10',
 'Множественная опухоль(4) - Развернутый клинический диагноз',
 'Множественная опухоль(4) - Статус диагноза',
 'Множественная опухоль(4) - Дата установления диагноза',
 'Множественная опухоль(4) - Основание диагноза',
 'Множественная опухоль(4) - Метод подтверждения диагноза',
 'Множественная опухоль(4) - Признак основной опухоли',
 'Множественная опухоль(4) - Сторона поражения',
 'Множественная опухоль(4) - Морфологический тип опухоли',
 'Множественная опухоль(4) - Обстоятельства выявления опухоли',
 'Множественная опухоль(4) - Стадия TNM',
 'Множественная опухоль(4) - pT',
 'Множественная опухоль(4) - pN',
 'Множественная опухоль(4) - pM',
 'Множественная опухоль(4) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(4) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(4) - Периневральная инвазия (Pn)',
 'Множественная опухоль(4) - Венозная инвазия (V)',
 'Множественная опухоль(4) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.24',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.25',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.26',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.27',
 'Множественная опухоль(4) - cT',
 'Множественная опухоль(4) - cN',
 'Множественная опухоль(4) - cM',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.28',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.29',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.30',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.31',
 'Множественная опухоль(4) - Стадия опухолевого процесса',
 'Множественная опухоль(4) - Клиническая группа',
 'ЦАОП',
 'ЦАОП_id',
 'ЦАОП_ОМС',
 'ЦАОП_в']

In [92]:
from transliterate import translit

list_of_eng_cols_to_db = []
for _ in total_data_caop.columns.tolist():
    list_of_eng_cols_to_db.append(translit(_, reversed=True))

In [94]:
db_col_list = [s.replace("'", "") for s in list_of_eng_cols_to_db]
db_col_list

['tip priema',
 'obschaja informatsija - ID protokola',
 'obschaja informatsija - ID patsienta',
 'obschaja informatsija - Data i vremja podpisanija protokola',
 'obschaja informatsija - FIOpatsienta',
 'obschaja informatsija - DR patsienta',
 'obschaja informatsija - OMS patsienta',
 'obschaja informatsija - FIO vracha',
 'obschaja informatsija - Dolzhnost vracha',
 'obschaja informatsija - MO priema',
 'obschaja informatsija - Strukturnoe podrazdelenie',
 'obschaja informatsija - Filial',
 'obschaja informatsija - Otdelenie',
 'obschaja informatsija - Spetsializatsija vracha',
 'Prikreplenie - ID uchastka prikreplenija v MU priema',
 'Prikreplenie - Uchastok prikreplenija v MU priema',
 'Prikreplenie - Prikreplenija v drugih MO',
 'Anamnez',
 'Zhaloby (predjavljaet/ne predjavljaet)',
 'Zhaloby',
 'Opisanie zhalob',
 'Schitaet sebja bolnym s ',
 'Obschee sostojanie',
 'Uroven soznanija',
 'Nervno-psihicheskij status',
 'NPS inoe',
 'Sostojanie patsienta (ECOG)',
 'Temperatura tela',
 

In [ ]:
'tip priema',
'obschaja informatsija - ID protokola',
'obschaja informatsija - ID patsienta',
'obschaja informatsija - Data i vremja podpisanija protokola',
'obschaja informatsija - FIOpatsienta',
'obschaja informatsija - DR patsienta',
'obschaja informatsija - OMS patsienta',
'obschaja informatsija - FIO vracha',
'obschaja informatsija - Dolzhnost vracha',
'obschaja informatsija - MO priema',
'obschaja informatsija - Strukturnoe podrazdelenie',
'obschaja informatsija - Filial',
'obschaja informatsija - Otdelenie',
'obschaja informatsija - Spetsializatsija vracha',
'Prikreplenie - ID uchastka prikreplenija v MU priema',
'Prikreplenie - Uchastok prikreplenija v MU priema',
'Prikreplenie - Prikreplenija v drugih MO',
'Anamnez',
'Zhaloby (predjavljaet/ne predjavljaet)',
'Zhaloby',
'Opisanie zhalob',
'Schitaet sebja bolnym s ',
'Obschee sostojanie',
'Uroven soznanija',
'Nervno-psihicheskij status',
'NPS inoe',
'Sostojanie patsienta (ECOG)',
'Temperatura tela',
'Lokalnyj status',
'Napravitelnyj diagnoz - MKB-10',
'Napravitelnyj diagnoz - Kod mkb-10',
'Diagnoz - MKB-10 rasshifrovka',
'Diagnoz - Kod MKB-10',
'Diagnoz - Razvernutyj klinicheskij diagnoz',
'Diagnoz - Data ustanvoki diagnoza',
'Diagnoz - Status diagnoza',
'Diagnoz - Pervichno-mnozhestvennaja opuhol (da/net)',
'Diagnoz - Metod podtverzhdenija diagnoza',
'Diagnoz - Klinicheskaja gruppa',
'Diagnoz - Osnovanie diagnoza',
'Prochie rekomendatsii',
'Novyj diagnoz - MKB-10',
'Novyj diagnoz - Kod MKB-10',
'Novyj diagnoz - Data ustanovlenija novogo diagnoza',
'Novyj diagnoz - Status novogo diagnoza',
'Edinichnaja opuhol - Storona porazhenija',
'Diagnoz - Morfologicheskij tip opuholi',
'Edinichnaja opuhol - Ostojatelstva vyjavlenija opuholi',
'Edinichnaja opuhol - Stadija TNM',
'Edinichnaja opuhol - Klassifikatsija ostatochnoj opuholi (R)',
'Edinichnaja opuhol - Gistopatologicheskaja stepen zlokachestvennosti (G)',
'Edinichnaja opuhol - Invazija v limfaticheskie sosudy (L)',
'Edinichnaja opuhol - Venoznaja invazija (V)',
'Edinichnaja opuhol - Perinevralnaja invazija (Pn)',
'Edinichnaja opuhol - Lokalizatsija otdalennyh metastazov',
'Edinichnaja opuhol - Klinicheskaja gruppa',
'Edinichnaja opuhol - Stadija opuholevogo protsessa',
'Biopsija - Trebuetsja vzjatie biopsii?',
'Biopsija - Prichina, po kotoroj vzjatie biopsii ne trebuetsja',
'Biopsija - Data provedenija biopsii',
'Biopsija - Otsutstvie informatsii o date provedenija biopsii',
'Biopsija - Gde provedena biopsija',
'Biopsija - Gde budet provoditsja biopsija (tip MO)',
'Biopsija - Gde budet provoditsja biopsija (MO)',
'Biopsija - Otsutstvie informatsii gde provedena biopsija',
'Onkokonsilium - Trebuetsja provedenie onkokonsiliuma? (true)',
'Onkokonsilium - Planovaja data provedenija onkokonsiliuma',
'Onkokonsilium - Mesto provedenija onkokonsiliuma',
'Spetsializirovannoe lechenie - Trebuetsja napravlenie na spetsializirovannoe lechenie',
'Spetsializirovannoe lechenie - Hirurgicheskoe lechenie',
'Spetsializirovannoe lechenie - Mesto provedenija hirurgicheskogo lechenija',
'Spetsializirovannoe lechenie - Luchevoe lechenie',
'Spetsializirovannoe lechenie - Lekarstvennoe lechenie',
'Spetsializirovannoe lechenie - Gde budet provoditsja lekarstvennoe lechenie?',
'Spetsializirovannoe lechenie - Mesto provedenija lechenija',
'Spetsializirovannoe lechenie - Planovaja data provedenija lekarstvennogo lechenija',
'Spetsializirovannoe lechenie - Trebuetsja naznachenie lekarstvennoj terapii?',
'Spetsializirovannoe lechenie - MNN/TN preparata',
'Spetsializirovannoe lechenie - Dozirovka',
'Dokumenty na prieme Spravki i napravlenija - Tip dokumenta',
'Dokumenty na prieme Spravki i napravlenija - Instrumentalnye obsledovanija',
'Dokumenty na prieme Spravki i napravlenija - Laboratornye issledovanija',
'Dokumenty na prieme Spravki i napravlenija - Konsultatsija spetsialista',
'Dokumenty na prieme Spravki i napravlenija - Gospitalizatsija',
'Dokumenty na prieme Spravki i napravlenija - Tsel vydachi',
'Vlozhennyj diagnoz na pervichnom prieme (1) - MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (1) - Kod MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (1) - Razvernutyj klinicheskij diagnoz',
'Vlozhennyj diagnoz na pervichnom prieme (1) - Metod podtverzhdenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (1) - Data ustanovlenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (1) - Klinicheskaja gruppa',
'Vlozhennyj diagnoz na pervichnom prieme (1) - Status diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (2) - MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (2) - Kod MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (2) - Razvernutyj klinicheskij diagnoz',
'Vlozhennyj diagnoz na pervichnom prieme (2) - Metod podtverzhdenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (2) - Data ustanovlenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (2) - Klinicheskaja gruppa',
'Vlozhennyj diagnoz na pervichnom prieme (2) - Status diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (3) - MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (3) - Kod MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (3) - Razvernutyj klinicheskij diagnoz',
'Vlozhennyj diagnoz na pervichnom prieme (3) - Metod podtverzhdenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (3) - Data ustanovlenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (3) - Klinicheskaja gruppa',
'Vlozhennyj diagnoz na pervichnom prieme (3) - Status diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (4) - MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (4) - Kod MKB-10',
'Vlozhennyj diagnoz na pervichnom prieme (4) - Razvernutyj klinicheskij diagnoz',
'Vlozhennyj diagnoz na pervichnom prieme (4) - Metod podtverzhdenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (4) - Data ustanovlenija diagnoza',
'Vlozhennyj diagnoz na pervichnom prieme (4) - Klinicheskaja gruppa',
'Vlozhennyj diagnoz na pervichnom prieme (4) - Status diagnoza',
'Edinichnaja opuhol - cT',
'Edinichnaja opuhol - cN',
'Edinichnaja opuhol - cM',
'Edinichnaja opuhol - pT',
'Edinichnaja opuhol - pN',
'Edinichnaja opuhol - pM',
'Mnozhestvennaja opuhol(1) - MKB-10',
'Mnozhestvennaja opuhol(1) - Kod MKB-10',
'Mnozhestvennaja opuhol(1) - Razvernutyj klinicheskij diagnoz',
'Mnozhestvennaja opuhol(1) - Status diagnoza',
'Mnozhestvennaja opuhol(1) - Data ustanovlenija diagnoza',
'Mnozhestvennaja opuhol(1) - Osnovanie diagnoza',
'Mnozhestvennaja opuhol(1) - Metod podtverzhdenija diagnoza',
'Mnozhestvennaja opuhol(1) - Priznak osnovnoj opuholi',
'Mnozhestvennaja opuhol(1) - Storona porazhenija',
'Mnozhestvennaja opuhol(1) - Morfologicheskij tip opuholi',
'Mnozhestvennaja opuhol(1) - Obstojatelstva vyjavlenija opuholi',
'Mnozhestvennaja opuhol(1) - Stadija TNM',
'Mnozhestvennaja opuhol(1) - pT',
'Mnozhestvennaja opuhol(1) - pN',
'Mnozhestvennaja opuhol(1) - pM',
'Mnozhestvennaja opuhol(1) - Klassifikatsija ostatochnoj opuholi (R)',
'Mnozhestvennaja opuhol(1) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
'Mnozhestvennaja opuhol(1) - Perinevralnaja invazija (Pn)',
'Mnozhestvennaja opuhol(1) - Venoznaja invazija (V)',
'Mnozhestvennaja opuhol(1) - Invazija v limfaticheskie sosudy (L)',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.1',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.2',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.3',
'Mnozhestvennaja opuhol(1) - cT',
'Mnozhestvennaja opuhol(1) - cN',
'Mnozhestvennaja opuhol(1) - cM',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.4',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.5',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.6',
'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.7',
'Mnozhestvennaja opuhol(1) - Stadija opuholevogo protsessa',
'Mnozhestvennaja opuhol(1) - Klinicheskaja gruppa',
'Mnozhestvennaja opuhol(2) - MKB-10',
'Mnozhestvennaja opuhol(2) - Kod MKB-10',
'Mnozhestvennaja opuhol(2) - Razvernutyj klinicheskij diagnoz',
'Mnozhestvennaja opuhol(2) - Status diagnoza',
'Mnozhestvennaja opuhol(2) - Data ustanovlenija diagnoza',
'Mnozhestvennaja opuhol(2) - Osnovanie diagnoza',
'Mnozhestvennaja opuhol(2) - Metod podtverzhdenija diagnoza',
'Mnozhestvennaja opuhol(2) - Priznak osnovnoj opuholi',
'Mnozhestvennaja opuhol(2) - Storona porazhenija',
'Mnozhestvennaja opuhol(2) - Morfologicheskij tip opuholi',
'Mnozhestvennaja opuhol(2) - Obstojatelstva vyjavlenija opuholi',
'Mnozhestvennaja opuhol(2) - Stadija TNM',
'Mnozhestvennaja opuhol(2) - pT',
'Mnozhestvennaja opuhol(2) - pN',
'Mnozhestvennaja opuhol(2) - pM',
'Mnozhestvennaja opuhol(2) - Klassifikatsija ostatochnoj opuholi (R)',
'Mnozhestvennaja opuhol(2) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
'Mnozhestvennaja opuhol(2) - Perinevralnaja invazija (Pn)',
'Mnozhestvennaja opuhol(2) - Venoznaja invazija (V)',
'Mnozhestvennaja opuhol(2) - Invazija v limfaticheskie sosudy (L)',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.8',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.9',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.10',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.11',
'Mnozhestvennaja opuhol(2) - cT',
'Mnozhestvennaja opuhol(2) - cN',
'Mnozhestvennaja opuhol(2) - cM',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.12',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.13',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.14',
'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.15',
'Mnozhestvennaja opuhol(2) - Stadija opuholevogo protsessa',
'Mnozhestvennaja opuhol(2) - Klinicheskaja gruppa',
'Mnozhestvennaja opuhol(3) - MKB-10',
'Mnozhestvennaja opuhol(3) - Kod MKB-10',
'Mnozhestvennaja opuhol(3) - Razvernutyj klinicheskij diagnoz',
'Mnozhestvennaja opuhol(3) - Status diagnoza',
'Mnozhestvennaja opuhol(3) - Data ustanovlenija diagnoza',
'Mnozhestvennaja opuhol(3) - Osnovanie diagnoza',
'Mnozhestvennaja opuhol(3) - Metod podtverzhdenija diagnoza',
'Mnozhestvennaja opuhol(3) - Priznak osnovnoj opuholi',
'Mnozhestvennaja opuhol(3) - Storona porazhenija',
'Mnozhestvennaja opuhol(3) - Morfologicheskij tip opuholi',
'Mnozhestvennaja opuhol(3) - Obstojatelstva vyjavlenija opuholi',
'Mnozhestvennaja opuhol(3) - Stadija TNM',
'Mnozhestvennaja opuhol(3) - pT',
'Mnozhestvennaja opuhol(3) - pN',
'Mnozhestvennaja opuhol(3) - pM',
'Mnozhestvennaja opuhol(3) - Klassifikatsija ostatochnoj opuholi (R)',
'Mnozhestvennaja opuhol(3) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
'Mnozhestvennaja opuhol(3) - Perinevralnaja invazija (Pn)',
'Mnozhestvennaja opuhol(3) - Venoznaja invazija (V)',
'Mnozhestvennaja opuhol(3) - Invazija v limfaticheskie sosudy (L)',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.16',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.17',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.18',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.19',
'Mnozhestvennaja opuhol(3) - cT',
'Mnozhestvennaja opuhol(3) - cN',
'Mnozhestvennaja opuhol(3) - cM',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.20',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.21',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.22',
'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.23',
'Mnozhestvennaja opuhol(3) - Stadija opuholevogo protsessa',
'Mnozhestvennaja opuhol(3) - Klinicheskaja gruppa',
'Mnozhestvennaja opuhol(4) - MKB-10',
'Mnozhestvennaja opuhol(4) - Kod MKB-10',
'Mnozhestvennaja opuhol(4) - Razvernutyj klinicheskij diagnoz',
'Mnozhestvennaja opuhol(4) - Status diagnoza',
'Mnozhestvennaja opuhol(4) - Data ustanovlenija diagnoza',
'Mnozhestvennaja opuhol(4) - Osnovanie diagnoza',
'Mnozhestvennaja opuhol(4) - Metod podtverzhdenija diagnoza',
'Mnozhestvennaja opuhol(4) - Priznak osnovnoj opuholi',
'Mnozhestvennaja opuhol(4) - Storona porazhenija',
'Mnozhestvennaja opuhol(4) - Morfologicheskij tip opuholi',
'Mnozhestvennaja opuhol(4) - Obstojatelstva vyjavlenija opuholi',
'Mnozhestvennaja opuhol(4) - Stadija TNM',
'Mnozhestvennaja opuhol(4) - pT',
'Mnozhestvennaja opuhol(4) - pN',
'Mnozhestvennaja opuhol(4) - pM',
'Mnozhestvennaja opuhol(4) - Klassifikatsija ostatochnoj opuholi (R)',
'Mnozhestvennaja opuhol(4) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
'Mnozhestvennaja opuhol(4) - Perinevralnaja invazija (Pn)',
'Mnozhestvennaja opuhol(4) - Venoznaja invazija (V)',
'Mnozhestvennaja opuhol(4) - Invazija v limfaticheskie sosudy (L)',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.24',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.25',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.26',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.27',
'Mnozhestvennaja opuhol(4) - cT',
'Mnozhestvennaja opuhol(4) - cN',
'Mnozhestvennaja opuhol(4) - cM',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.28',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.29',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.30',
'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.31',
'Mnozhestvennaja opuhol(4) - Stadija opuholevogo protsessa',
'Mnozhestvennaja opuhol(4) - Klinicheskaja gruppa',
'TsAOP',
'TsAOP_id',
'TsAOP_OMS',
'TsAOP_v'

In [ ]:
['obschaja_informatsija_-_id_protokola',
'tip_priema',
'obschaja_informatsija_-_id_patsienta',
'obschaja_informatsija_-_data_i_vremja_podpisanija_protokola',
'obschaja_informatsija_-_fiopatsienta',
'obschaja_informatsija_-_dr_patsienta',
'obschaja_informatsija_-_oms_patsienta',
'obschaja_informatsija_-_fio_vracha',
'obschaja_informatsija_-_dolzhnost_vracha',
'obschaja_informatsija_-_mo_priema',
'obschaja_informatsija_-_strukturnoe_podrazdelenie',
'obschaja_informatsija_-_filial',
'obschaja_informatsija_-_otdelenie',
'obschaja_informatsija_-_spetsializatsija_vracha',
'prikreplenie_-_id_uchastka_prikreplenija_v_mu_priema',
'prikreplenie_-_uchastok_prikreplenija_v_mu_priema',
'prikreplenie_-_prikreplenija_v_drugih_mo',
'anamnez',
'zhaloby_(predjavljaet/ne_predjavljaet)',
'zhaloby',
'opisanie_zhalob',
'schitaet_sebja_bolnym_s_',
'obschee_sostojanie',
'uroven_soznanija',
'nervno-psihicheskij_status',
'nps_inoe',
'sostojanie_patsienta_(ecog)',
'temperatura_tela',
'lokalnyj_status',
'napravitelnyj_diagnoz_-_mkb-10',
'napravitelnyj_diagnoz_-_kod_mkb-10',
'diagnoz_-_mkb-10_rasshifrovka',
'diagnoz_-_kod_mkb-10',
'diagnoz_-_razvernutyj_klinicheskij_diagnoz',
'diagnoz_-_data_ustanvoki_diagnoza',
'diagnoz_-_status_diagnoza',
'diagnoz_-_pervichno-mnozhestvennaja_opuhol_(da/net)',
'diagnoz_-_metod_podtverzhdenija_diagnoza',
'diagnoz_-_klinicheskaja_gruppa',
'diagnoz_-_osnovanie_diagnoza',
'prochie_rekomendatsii',
'novyj_diagnoz_-_mkb-10',
'novyj_diagnoz_-_kod_mkb-10',
'novyj_diagnoz_-_data_ustanovlenija_novogo_diagnoza',
'novyj_diagnoz_-_status_novogo_diagnoza',
'edinichnaja_opuhol_-_storona_porazhenija',
'diagnoz_-_morfologicheskij_tip_opuholi',
'edinichnaja_opuhol_-_ostojatelstva_vyjavlenija_opuholi',
'edinichnaja_opuhol_-_stadija_tnm',
'edinichnaja_opuhol_-_klassifikatsija_ostatochnoj_opuholi_(r)',
'edinichnaja_opuhol_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g)',
'edinichnaja_opuhol_-_invazija_v_limfaticheskie_sosudy_(l)',
'edinichnaja_opuhol_-_venoznaja_invazija_(v)',
'edinichnaja_opuhol_-_perinevralnaja_invazija_(pn)',
'edinichnaja_opuhol_-_lokalizatsija_otdalennyh_metastazov',
'edinichnaja_opuhol_-_klinicheskaja_gruppa',
'edinichnaja_opuhol_-_stadija_opuholevogo_protsessa',
'biopsija_-_trebuetsja_vzjatie_biopsii?',
'biopsija_-_prichina,_po_kotoroj_vzjatie_biopsii_ne_trebuetsja',
'biopsija_-_data_provedenija_biopsii',
'biopsija_-_otsutstvie_informatsii_o_date_provedenija_biopsii',
'biopsija_-_gde_provedena_biopsija',
'biopsija_-_gde_budet_provoditsja_biopsija_(tip_mo)',
'biopsija_-_gde_budet_provoditsja_biopsija_(mo)',
'biopsija_-_otsutstvie_informatsii_gde_provedena_biopsija',
'onkokonsilium_-_trebuetsja_provedenie_onkokonsiliuma?_(true)',
'onkokonsilium_-_planovaja_data_provedenija_onkokonsiliuma',
'onkokonsilium_-_mesto_provedenija_onkokonsiliuma',
'spetsializirovannoe_lechenie_-_trebuetsja_napravlenie_na_spetsializirovannoe_lechenie',
'spetsializirovannoe_lechenie_-_hirurgicheskoe_lechenie',
'spetsializirovannoe_lechenie_-_mesto_provedenija_hirurgicheskogo_lechenija',
'spetsializirovannoe_lechenie_-_luchevoe_lechenie',
'spetsializirovannoe_lechenie_-_lekarstvennoe_lechenie',
'spetsializirovannoe_lechenie_-_gde_budet_provoditsja_lekarstvennoe_lechenie?',
'spetsializirovannoe_lechenie_-_mesto_provedenija_lechenija',
'spetsializirovannoe_lechenie_-_planovaja_data_provedenija_lekarstvennogo_lechenija',
'spetsializirovannoe_lechenie_-_trebuetsja_naznachenie_lekarstvennoj_terapii?',
'spetsializirovannoe_lechenie_-_mnn/tn_preparata',
'spetsializirovannoe_lechenie_-_dozirovka',
'dokumenty_na_prieme_spravki_i_napravlenija_-_tip_dokumenta',
'dokumenty_na_prieme_spravki_i_napravlenija_-_instrumentalnye_obsledovanija',
'dokumenty_na_prieme_spravki_i_napravlenija_-_laboratornye_issledovanija',
'dokumenty_na_prieme_spravki_i_napravlenija_-_konsultatsija_spetsialista',
'dokumenty_na_prieme_spravki_i_napravlenija_-_gospitalizatsija',
'dokumenty_na_prieme_spravki_i_napravlenija_-_tsel_vydachi',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_kod_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_razvernutyj_klinicheskij_diagnoz',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_metod_podtverzhdenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_data_ustanovlenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_klinicheskaja_gruppa',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_status_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_kod_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_razvernutyj_klinicheskij_diagnoz',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_metod_podtverzhdenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_data_ustanovlenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_klinicheskaja_gruppa',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_status_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_kod_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_razvernutyj_klinicheskij_diagnoz',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_metod_podtverzhdenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_data_ustanovlenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_klinicheskaja_gruppa',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_status_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_kod_mkb-10',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_razvernutyj_klinicheskij_diagnoz',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_metod_podtverzhdenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_data_ustanovlenija_diagnoza',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_klinicheskaja_gruppa',
'vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_status_diagnoza',
'edinichnaja_opuhol_-_ct',
'edinichnaja_opuhol_-_cn',
'edinichnaja_opuhol_-_cm',
'edinichnaja_opuhol_-_pt',
'edinichnaja_opuhol_-_pn',
'edinichnaja_opuhol_-_pm',
'mnozhestvennaja_opuhol(1)_-_mkb-10',
'mnozhestvennaja_opuhol(1)_-_kod_mkb-10',
'mnozhestvennaja_opuhol(1)_-_razvernutyj_klinicheskij_diagnoz',
'mnozhestvennaja_opuhol(1)_-_status_diagnoza',
'mnozhestvennaja_opuhol(1)_-_data_ustanovlenija_diagnoza',
'mnozhestvennaja_opuhol(1)_-_osnovanie_diagnoza',
'mnozhestvennaja_opuhol(1)_-_metod_podtverzhdenija_diagnoza',
'mnozhestvennaja_opuhol(1)_-_priznak_osnovnoj_opuholi',
'mnozhestvennaja_opuhol(1)_-_storona_porazhenija',
'mnozhestvennaja_opuhol(1)_-_morfologicheskij_tip_opuholi',
'mnozhestvennaja_opuhol(1)_-_obstojatelstva_vyjavlenija_opuholi',
'mnozhestvennaja_opuhol(1)_-_stadija_tnm',
'mnozhestvennaja_opuhol(1)_-_pt',
'mnozhestvennaja_opuhol(1)_-_pn',
'mnozhestvennaja_opuhol(1)_-_pm',
'mnozhestvennaja_opuhol(1)_-_klassifikatsija_ostatochnoj_opuholi_(r)',
'mnozhestvennaja_opuhol(1)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g)',
'mnozhestvennaja_opuhol(1)_-_perinevralnaja_invazija_(pn)',
'mnozhestvennaja_opuhol(1)_-_venoznaja_invazija_(v)',
'mnozhestvennaja_opuhol(1)_-_invazija_v_limfaticheskie_sosudy_(l)',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.1',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.2',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.3',
'mnozhestvennaja_opuhol(1)_-_ct',
'mnozhestvennaja_opuhol(1)_-_cn',
'mnozhestvennaja_opuhol(1)_-_cm',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.4',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.5',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.6',
'mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.7',
'mnozhestvennaja_opuhol(1)_-_stadija_opuholevogo_protsessa',
'mnozhestvennaja_opuhol(1)_-_klinicheskaja_gruppa',
'mnozhestvennaja_opuhol(2)_-_mkb-10',
'mnozhestvennaja_opuhol(2)_-_kod_mkb-10',
'mnozhestvennaja_opuhol(2)_-_razvernutyj_klinicheskij_diagnoz',
'mnozhestvennaja_opuhol(2)_-_status_diagnoza',
'mnozhestvennaja_opuhol(2)_-_data_ustanovlenija_diagnoza',
'mnozhestvennaja_opuhol(2)_-_osnovanie_diagnoza',
'mnozhestvennaja_opuhol(2)_-_metod_podtverzhdenija_diagnoza',
'mnozhestvennaja_opuhol(2)_-_priznak_osnovnoj_opuholi',
'mnozhestvennaja_opuhol(2)_-_storona_porazhenija',
'mnozhestvennaja_opuhol(2)_-_morfologicheskij_tip_opuholi',
'mnozhestvennaja_opuhol(2)_-_obstojatelstva_vyjavlenija_opuholi',
'mnozhestvennaja_opuhol(2)_-_stadija_tnm',
'mnozhestvennaja_opuhol(2)_-_pt',
'mnozhestvennaja_opuhol(2)_-_pn',
'mnozhestvennaja_opuhol(2)_-_pm',
'mnozhestvennaja_opuhol(2)_-_klassifikatsija_ostatochnoj_opuholi_(r)',
'mnozhestvennaja_opuhol(2)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g)',
'mnozhestvennaja_opuhol(2)_-_perinevralnaja_invazija_(pn)',
'mnozhestvennaja_opuhol(2)_-_venoznaja_invazija_(v)',
'mnozhestvennaja_opuhol(2)_-_invazija_v_limfaticheskie_sosudy_(l)',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.8',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.9',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.10',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.11',
'mnozhestvennaja_opuhol(2)_-_ct',
'mnozhestvennaja_opuhol(2)_-_cn',
'mnozhestvennaja_opuhol(2)_-_cm',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.12',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.13',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.14',
'mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.15',
'mnozhestvennaja_opuhol(2)_-_stadija_opuholevogo_protsessa',
'mnozhestvennaja_opuhol(2)_-_klinicheskaja_gruppa',
'mnozhestvennaja_opuhol(3)_-_mkb-10',
'mnozhestvennaja_opuhol(3)_-_kod_mkb-10',
'mnozhestvennaja_opuhol(3)_-_razvernutyj_klinicheskij_diagnoz',
'mnozhestvennaja_opuhol(3)_-_status_diagnoza',
'mnozhestvennaja_opuhol(3)_-_data_ustanovlenija_diagnoza',
'mnozhestvennaja_opuhol(3)_-_osnovanie_diagnoza',
'mnozhestvennaja_opuhol(3)_-_metod_podtverzhdenija_diagnoza',
'mnozhestvennaja_opuhol(3)_-_priznak_osnovnoj_opuholi',
'mnozhestvennaja_opuhol(3)_-_storona_porazhenija',
'mnozhestvennaja_opuhol(3)_-_morfologicheskij_tip_opuholi',
'mnozhestvennaja_opuhol(3)_-_obstojatelstva_vyjavlenija_opuholi',
'mnozhestvennaja_opuhol(3)_-_stadija_tnm',
'mnozhestvennaja_opuhol(3)_-_pt',
'mnozhestvennaja_opuhol(3)_-_pn',
'mnozhestvennaja_opuhol(3)_-_pm',
'mnozhestvennaja_opuhol(3)_-_klassifikatsija_ostatochnoj_opuholi_(r)',
'mnozhestvennaja_opuhol(3)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g)',
'mnozhestvennaja_opuhol(3)_-_perinevralnaja_invazija_(pn)',
'mnozhestvennaja_opuhol(3)_-_venoznaja_invazija_(v)',
'mnozhestvennaja_opuhol(3)_-_invazija_v_limfaticheskie_sosudy_(l)',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.16',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.17',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.18',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.19',
'mnozhestvennaja_opuhol(3)_-_ct',
'mnozhestvennaja_opuhol(3)_-_cn',
'mnozhestvennaja_opuhol(3)_-_cm',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.20',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.21',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.22',
'mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.23',
'mnozhestvennaja_opuhol(3)_-_stadija_opuholevogo_protsessa',
'mnozhestvennaja_opuhol(3)_-_klinicheskaja_gruppa',
'mnozhestvennaja_opuhol(4)_-_mkb-10',
'mnozhestvennaja_opuhol(4)_-_kod_mkb-10',
'mnozhestvennaja_opuhol(4)_-_razvernutyj_klinicheskij_diagnoz',
'mnozhestvennaja_opuhol(4)_-_status_diagnoza',
'mnozhestvennaja_opuhol(4)_-_data_ustanovlenija_diagnoza',
'mnozhestvennaja_opuhol(4)_-_osnovanie_diagnoza',
'mnozhestvennaja_opuhol(4)_-_metod_podtverzhdenija_diagnoza',
'mnozhestvennaja_opuhol(4)_-_priznak_osnovnoj_opuholi',
'mnozhestvennaja_opuhol(4)_-_storona_porazhenija',
'mnozhestvennaja_opuhol(4)_-_morfologicheskij_tip_opuholi',
'mnozhestvennaja_opuhol(4)_-_obstojatelstva_vyjavlenija_opuholi',
'mnozhestvennaja_opuhol(4)_-_stadija_tnm',
'mnozhestvennaja_opuhol(4)_-_pt',
'mnozhestvennaja_opuhol(4)_-_pn',
'mnozhestvennaja_opuhol(4)_-_pm',
'mnozhestvennaja_opuhol(4)_-_klassifikatsija_ostatochnoj_opuholi_(r)',
'mnozhestvennaja_opuhol(4)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g)',
'mnozhestvennaja_opuhol(4)_-_perinevralnaja_invazija_(pn)',
'mnozhestvennaja_opuhol(4)_-_venoznaja_invazija_(v)',
'mnozhestvennaja_opuhol(4)_-_invazija_v_limfaticheskie_sosudy_(l)',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.24',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.25',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.26',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.27',
'mnozhestvennaja_opuhol(4)_-_ct',
'mnozhestvennaja_opuhol(4)_-_cn',
'mnozhestvennaja_opuhol(4)_-_cm',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.28',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.29',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.30',
'mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.31',
'mnozhestvennaja_opuhol(4)_-_stadija_opuholevogo_protsessa',
'mnozhestvennaja_opuhol(4)_-_klinicheskaja_gruppa',
'tsaop',
'tsaop_id',
'tsaop_oms',
'tsaop_v']

In [100]:
db_col_list_lower = list(map(lambda x: x.lower(), db_col_list))
db_col_list_lower_drop_space = list(map(lambda x: x.replace(' ','_'), db_col_list_lower))
db_col_list_lower_drop_space

['obschaja_informatsija_-_id_protokola',
 'tip_priema',
 'obschaja_informatsija_-_id_patsienta',
 'obschaja_informatsija_-_data_i_vremja_podpisanija_protokola',
 'obschaja_informatsija_-_fiopatsienta',
 'obschaja_informatsija_-_dr_patsienta',
 'obschaja_informatsija_-_oms_patsienta',
 'obschaja_informatsija_-_fio_vracha',
 'obschaja_informatsija_-_dolzhnost_vracha',
 'obschaja_informatsija_-_mo_priema',
 'obschaja_informatsija_-_strukturnoe_podrazdelenie',
 'obschaja_informatsija_-_filial',
 'obschaja_informatsija_-_otdelenie',
 'obschaja_informatsija_-_spetsializatsija_vracha',
 'prikreplenie_-_id_uchastka_prikreplenija_v_mu_priema',
 'prikreplenie_-_uchastok_prikreplenija_v_mu_priema',
 'prikreplenie_-_prikreplenija_v_drugih_mo',
 'anamnez',
 'zhaloby_(predjavljaet/ne_predjavljaet)',
 'zhaloby',
 'opisanie_zhalob',
 'schitaet_sebja_bolnym_s_',
 'obschee_sostojanie',
 'uroven_soznanija',
 'nervno-psihicheskij_status',
 'nps_inoe',
 'sostojanie_patsienta_(ecog)',
 'temperatura_tela',
 

In [1]:
db_col_list = [ 'obschaja informatsija - ID protokola',
 'tip priema',
 'obschaja informatsija - ID patsienta',
 'obschaja informatsija - Data i vremja podpisanija protokola',
 'obschaja informatsija - FIOpatsienta',
 'obschaja informatsija - DR patsienta',
 'obschaja informatsija - OMS patsienta',
 'obschaja informatsija - FIO vracha',
 'obschaja informatsija - Dolzhnost vracha',
 'obschaja informatsija - MO priema',
 'obschaja informatsija - Strukturnoe podrazdelenie',
 'obschaja informatsija - Filial',
 'obschaja informatsija - Otdelenie',
 'obschaja informatsija - Spetsializatsija vracha',
 'Prikreplenie - ID uchastka prikreplenija v MU priema',
 'Prikreplenie - Uchastok prikreplenija v MU priema',
 'Prikreplenie - Prikreplenija v drugih MO',
 'Anamnez',
 'Zhaloby (predjavljaet/ne predjavljaet)',
 'Zhaloby',
 'Opisanie zhalob',
 'Schitaet sebja bolnym s ',
 'Obschee sostojanie',
 'Uroven soznanija',
 'Nervno-psihicheskij status',
 'NPS inoe',
 'Sostojanie patsienta (ECOG)',
 'Temperatura tela',
 'Lokalnyj status',
 'Napravitelnyj diagnoz - MKB-10',
 'Napravitelnyj diagnoz - Kod mkb-10',
 'Diagnoz - MKB-10 rasshifrovka',
 'Diagnoz - Kod MKB-10',
 'Diagnoz - Razvernutyj klinicheskij diagnoz',
 'Diagnoz - Data ustanvoki diagnoza',
 'Diagnoz - Status diagnoza',
 'Diagnoz - Pervichno-mnozhestvennaja opuhol (da/net)',
 'Diagnoz - Metod podtverzhdenija diagnoza',
 'Diagnoz - Klinicheskaja gruppa',
 'Diagnoz - Osnovanie diagnoza',
 'Prochie rekomendatsii',
 'Novyj diagnoz - MKB-10',
 'Novyj diagnoz - Kod MKB-10',
 'Novyj diagnoz - Data ustanovlenija novogo diagnoza',
 'Novyj diagnoz - Status novogo diagnoza',
 'Edinichnaja opuhol - Storona porazhenija',
 'Diagnoz - Morfologicheskij tip opuholi',
 'Edinichnaja opuhol - Ostojatelstva vyjavlenija opuholi',
 'Edinichnaja opuhol - Stadija TNM',
 'Edinichnaja opuhol - Klassifikatsija ostatochnoj opuholi (R)',
 'Edinichnaja opuhol - Gistopatologicheskaja stepen zlokachestvennosti (G)',
 'Edinichnaja opuhol - Invazija v limfaticheskie sosudy (L)',
 'Edinichnaja opuhol - Venoznaja invazija (V)',
 'Edinichnaja opuhol - Perinevralnaja invazija (Pn)',
 'Edinichnaja opuhol - Lokalizatsija otdalennyh metastazov',
 'Edinichnaja opuhol - Klinicheskaja gruppa',
 'Edinichnaja opuhol - Stadija opuholevogo protsessa',
 'Biopsija - Trebuetsja vzjatie biopsii?',
 'Biopsija - Prichina, po kotoroj vzjatie biopsii ne trebuetsja',
 'Biopsija - Data provedenija biopsii',
 'Biopsija - Otsutstvie informatsii o date provedenija biopsii',
 'Biopsija - Gde provedena biopsija',
 'Biopsija - Gde budet provoditsja biopsija (tip MO)',
 'Biopsija - Gde budet provoditsja biopsija (MO)',
 'Biopsija - Otsutstvie informatsii gde provedena biopsija',
 'Onkokonsilium - Trebuetsja provedenie onkokonsiliuma? (true)',
 'Onkokonsilium - Planovaja data provedenija onkokonsiliuma',
 'Onkokonsilium - Mesto provedenija onkokonsiliuma',
 'Spetsializirovannoe lechenie - Trebuetsja napravlenie na spetsializirovannoe lechenie',
 'Spetsializirovannoe lechenie - Hirurgicheskoe lechenie',
 'Spetsializirovannoe lechenie - Mesto provedenija hirurgicheskogo lechenija',
 'Spetsializirovannoe lechenie - Luchevoe lechenie',
 'Spetsializirovannoe lechenie - Lekarstvennoe lechenie',
 'Spetsializirovannoe lechenie - Gde budet provoditsja lekarstvennoe lechenie?',
 'Spetsializirovannoe lechenie - Mesto provedenija lechenija',
 'Spetsializirovannoe lechenie - Planovaja data provedenija lekarstvennogo lechenija',
 'Spetsializirovannoe lechenie - Trebuetsja naznachenie lekarstvennoj terapii?',
 'Spetsializirovannoe lechenie - MNN/TN preparata',
 'Spetsializirovannoe lechenie - Dozirovka',
 'Dokumenty na prieme Spravki i napravlenija - Tip dokumenta',
 'Dokumenty na prieme Spravki i napravlenija - Instrumentalnye obsledovanija',
 'Dokumenty na prieme Spravki i napravlenija - Laboratornye issledovanija',
 'Dokumenty na prieme Spravki i napravlenija - Konsultatsija spetsialista',
 'Dokumenty na prieme Spravki i napravlenija - Gospitalizatsija',
 'Dokumenty na prieme Spravki i napravlenija - Tsel vydachi',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - Kod MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - Razvernutyj klinicheskij diagnoz',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - Metod podtverzhdenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - Data ustanovlenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - Klinicheskaja gruppa',
 'Vlozhennyj diagnoz na pervichnom prieme (1) - Status diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - Kod MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - Razvernutyj klinicheskij diagnoz',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - Metod podtverzhdenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - Data ustanovlenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - Klinicheskaja gruppa',
 'Vlozhennyj diagnoz na pervichnom prieme (2) - Status diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - Kod MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - Razvernutyj klinicheskij diagnoz',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - Metod podtverzhdenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - Data ustanovlenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - Klinicheskaja gruppa',
 'Vlozhennyj diagnoz na pervichnom prieme (3) - Status diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - Kod MKB-10',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - Razvernutyj klinicheskij diagnoz',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - Metod podtverzhdenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - Data ustanovlenija diagnoza',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - Klinicheskaja gruppa',
 'Vlozhennyj diagnoz na pervichnom prieme (4) - Status diagnoza',
 'Edinichnaja opuhol - cT',
 'Edinichnaja opuhol - cN',
 'Edinichnaja opuhol - cM',
 'Edinichnaja opuhol - pT',
 'Edinichnaja opuhol - pN',
 'Edinichnaja opuhol - pM',
 'Mnozhestvennaja opuhol(1) - MKB-10',
 'Mnozhestvennaja opuhol(1) - Kod MKB-10',
 'Mnozhestvennaja opuhol(1) - Razvernutyj klinicheskij diagnoz',
 'Mnozhestvennaja opuhol(1) - Status diagnoza',
 'Mnozhestvennaja opuhol(1) - Data ustanovlenija diagnoza',
 'Mnozhestvennaja opuhol(1) - Osnovanie diagnoza',
 'Mnozhestvennaja opuhol(1) - Metod podtverzhdenija diagnoza',
 'Mnozhestvennaja opuhol(1) - Priznak osnovnoj opuholi',
 'Mnozhestvennaja opuhol(1) - Storona porazhenija',
 'Mnozhestvennaja opuhol(1) - Morfologicheskij tip opuholi',
 'Mnozhestvennaja opuhol(1) - Obstojatelstva vyjavlenija opuholi',
 'Mnozhestvennaja opuhol(1) - Stadija TNM',
 'Mnozhestvennaja opuhol(1) - pT',
 'Mnozhestvennaja opuhol(1) - pN',
 'Mnozhestvennaja opuhol(1) - pM',
 'Mnozhestvennaja opuhol(1) - Klassifikatsija ostatochnoj opuholi (R)',
 'Mnozhestvennaja opuhol(1) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
 'Mnozhestvennaja opuhol(1) - Perinevralnaja invazija (Pn)',
 'Mnozhestvennaja opuhol(1) - Venoznaja invazija (V)',
 'Mnozhestvennaja opuhol(1) - Invazija v limfaticheskie sosudy (L)',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.1',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.2',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.3',
 'Mnozhestvennaja opuhol(1) - cT',
 'Mnozhestvennaja opuhol(1) - cN',
 'Mnozhestvennaja opuhol(1) - cM',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.4',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.5',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.6',
 'Mnozhestvennaja opuhol(1) - Lokalizatsija otdalennyh metastazov.7',
 'Mnozhestvennaja opuhol(1) - Stadija opuholevogo protsessa',
 'Mnozhestvennaja opuhol(1) - Klinicheskaja gruppa',
 'Mnozhestvennaja opuhol(2) - MKB-10',
 'Mnozhestvennaja opuhol(2) - Kod MKB-10',
 'Mnozhestvennaja opuhol(2) - Razvernutyj klinicheskij diagnoz',
 'Mnozhestvennaja opuhol(2) - Status diagnoza',
 'Mnozhestvennaja opuhol(2) - Data ustanovlenija diagnoza',
 'Mnozhestvennaja opuhol(2) - Osnovanie diagnoza',
 'Mnozhestvennaja opuhol(2) - Metod podtverzhdenija diagnoza',
 'Mnozhestvennaja opuhol(2) - Priznak osnovnoj opuholi',
 'Mnozhestvennaja opuhol(2) - Storona porazhenija',
 'Mnozhestvennaja opuhol(2) - Morfologicheskij tip opuholi',
 'Mnozhestvennaja opuhol(2) - Obstojatelstva vyjavlenija opuholi',
 'Mnozhestvennaja opuhol(2) - Stadija TNM',
 'Mnozhestvennaja opuhol(2) - pT',
 'Mnozhestvennaja opuhol(2) - pN',
 'Mnozhestvennaja opuhol(2) - pM',
 'Mnozhestvennaja opuhol(2) - Klassifikatsija ostatochnoj opuholi (R)',
 'Mnozhestvennaja opuhol(2) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
 'Mnozhestvennaja opuhol(2) - Perinevralnaja invazija (Pn)',
 'Mnozhestvennaja opuhol(2) - Venoznaja invazija (V)',
 'Mnozhestvennaja opuhol(2) - Invazija v limfaticheskie sosudy (L)',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.8',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.9',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.10',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.11',
 'Mnozhestvennaja opuhol(2) - cT',
 'Mnozhestvennaja opuhol(2) - cN',
 'Mnozhestvennaja opuhol(2) - cM',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.12',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.13',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.14',
 'Mnozhestvennaja opuhol(2) - Lokalizatsija otdalennyh metastazov.15',
 'Mnozhestvennaja opuhol(2) - Stadija opuholevogo protsessa',
 'Mnozhestvennaja opuhol(2) - Klinicheskaja gruppa',
 'Mnozhestvennaja opuhol(3) - MKB-10',
 'Mnozhestvennaja opuhol(3) - Kod MKB-10',
 'Mnozhestvennaja opuhol(3) - Razvernutyj klinicheskij diagnoz',
 'Mnozhestvennaja opuhol(3) - Status diagnoza',
 'Mnozhestvennaja opuhol(3) - Data ustanovlenija diagnoza',
 'Mnozhestvennaja opuhol(3) - Osnovanie diagnoza',
 'Mnozhestvennaja opuhol(3) - Metod podtverzhdenija diagnoza',
 'Mnozhestvennaja opuhol(3) - Priznak osnovnoj opuholi',
 'Mnozhestvennaja opuhol(3) - Storona porazhenija',
 'Mnozhestvennaja opuhol(3) - Morfologicheskij tip opuholi',
 'Mnozhestvennaja opuhol(3) - Obstojatelstva vyjavlenija opuholi',
 'Mnozhestvennaja opuhol(3) - Stadija TNM',
 'Mnozhestvennaja opuhol(3) - pT',
 'Mnozhestvennaja opuhol(3) - pN',
 'Mnozhestvennaja opuhol(3) - pM',
 'Mnozhestvennaja opuhol(3) - Klassifikatsija ostatochnoj opuholi (R)',
 'Mnozhestvennaja opuhol(3) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
 'Mnozhestvennaja opuhol(3) - Perinevralnaja invazija (Pn)',
 'Mnozhestvennaja opuhol(3) - Venoznaja invazija (V)',
 'Mnozhestvennaja opuhol(3) - Invazija v limfaticheskie sosudy (L)',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.16',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.17',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.18',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.19',
 'Mnozhestvennaja opuhol(3) - cT',
 'Mnozhestvennaja opuhol(3) - cN',
 'Mnozhestvennaja opuhol(3) - cM',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.20',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.21',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.22',
 'Mnozhestvennaja opuhol(3) - Lokalizatsija otdalennyh metastazov.23',
 'Mnozhestvennaja opuhol(3) - Stadija opuholevogo protsessa',
 'Mnozhestvennaja opuhol(3) - Klinicheskaja gruppa',
 'Mnozhestvennaja opuhol(4) - MKB-10',
 'Mnozhestvennaja opuhol(4) - Kod MKB-10',
 'Mnozhestvennaja opuhol(4) - Razvernutyj klinicheskij diagnoz',
 'Mnozhestvennaja opuhol(4) - Status diagnoza',
 'Mnozhestvennaja opuhol(4) - Data ustanovlenija diagnoza',
 'Mnozhestvennaja opuhol(4) - Osnovanie diagnoza',
 'Mnozhestvennaja opuhol(4) - Metod podtverzhdenija diagnoza',
 'Mnozhestvennaja opuhol(4) - Priznak osnovnoj opuholi',
 'Mnozhestvennaja opuhol(4) - Storona porazhenija',
 'Mnozhestvennaja opuhol(4) - Morfologicheskij tip opuholi',
 'Mnozhestvennaja opuhol(4) - Obstojatelstva vyjavlenija opuholi',
 'Mnozhestvennaja opuhol(4) - Stadija TNM',
 'Mnozhestvennaja opuhol(4) - pT',
 'Mnozhestvennaja opuhol(4) - pN',
 'Mnozhestvennaja opuhol(4) - pM',
 'Mnozhestvennaja opuhol(4) - Klassifikatsija ostatochnoj opuholi (R)',
 'Mnozhestvennaja opuhol(4) - Gistopatologicheskaja stepen zlokachestvennosti (G)',
 'Mnozhestvennaja opuhol(4) - Perinevralnaja invazija (Pn)',
 'Mnozhestvennaja opuhol(4) - Venoznaja invazija (V)',
 'Mnozhestvennaja opuhol(4) - Invazija v limfaticheskie sosudy (L)',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.24',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.25',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.26',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.27',
 'Mnozhestvennaja opuhol(4) - cT',
 'Mnozhestvennaja opuhol(4) - cN',
 'Mnozhestvennaja opuhol(4) - cM',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.28',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.29',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.30',
 'Mnozhestvennaja opuhol(4) - Lokalizatsija otdalennyh metastazov.31',
 'Mnozhestvennaja opuhol(4) - Stadija opuholevogo protsessa',
 'Mnozhestvennaja opuhol(4) - Klinicheskaja gruppa',
 'TsAOP',
 'TsAOP_id',
 'TsAOP_OMS',
 'TsAOP_v']

In [2]:
len(db_col_list)

255

In [101]:
try:
    sqlite_connection = sqlite3.connect('test_sqlite_python.db')
    
    sqlite_create_table_query = '''CREATE TABLE test_sqlite_db (
                                inner_id INTEGER PRIMARY KEY,
                                tip_priema TEXT,
                                obschaja_informatsija_-_id_protokola TEXT,
                                bschaja_informatsija_-_id_patsienta TEXT,
                                obschaja_informatsija_-_data_i_vremja_podpisanija_protokola ТЕХТ,
                                obschaja_informatsija_-_fiopatsienta TEXT,
                                obschaja_informatsija_-_dr_patsienta ТЕХТ,
                                obschaja_informatsija_-_oms_patsienta TEXT,
                                obschaja_informatsija_-_fio_vracha TEXT,
                                obschaja_informatsija_-_dolzhnost_vracha TEXT,
                                obschaja_informatsija_-_mo_priema TEXT,
                                obschaja_informatsija_-_strukturnoe_podrazdelenie TEXT,
                                obschaja_informatsija_-_filial TEXT,
                                obschaja_informatsija_-_otdelenie TEXT,
                                obschaja_informatsija_-_spetsializatsija_vracha TEXT,
                                prikreplenie_-_id_uchastka_prikreplenija_v_mu_priema TEXT,
                                prikreplenie_-_uchastok_prikreplenija_v_mu_priema TEXT,
                                prikreplenie_-_prikreplenija_v_drugih_mo TEXT,
                                anamnez TEXT,
                                zhaloby_(predjavljaet/ne_predjavljaet) TEXT,
                                zhaloby TEXT,
                                opisanie_zhalob TEXT,
                                schitaet_sebja_bolnym_s_ TEXT,
                                obschee_sostojanie TEXT,
                                uroven_soznanija TEXT,
                                nervno-psihicheskij_status TEXT,
                                nps_inoe TEXT,
                                sostojanie_patsienta_(ecog) TEXT,
                                temperatura_tela TEXT,
                                lokalnyj_status TEXT,
                                napravitelnyj_diagnoz_-_mkb-10 TEXT,
                                napravitelnyj_diagnoz_-_kod_mkb-10 TEXT,
                                diagnoz_-_mkb-10_rasshifrovka TEXT,
                                diagnoz_-_kod_mkb-10 TEXT,
                                diagnoz_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                diagnoz_-_data_ustanvoki_diagnoza TEXT,
                                diagnoz_-_status_diagnoza TEXT,
                                diagnoz_-_pervichno-mnozhestvennaja_opuhol_(da/net) TEXT,
                                diagnoz_-_metod_podtverzhdenija_diagnoza TEXT,
                                diagnoz_-_klinicheskaja_gruppa TEXT,
                                diagnoz_-_osnovanie_diagnoza TEXT,
                                prochie_rekomendatsii TEXT,
                                novyj_diagnoz_-_mkb-10 TEXT,
                                novyj_diagnoz_-_kod_mkb-10 TEXT,
                                novyj_diagnoz_-_data_ustanovlenija_novogo_diagnoza TEXT,
                                novyj_diagnoz_-_status_novogo_diagnoza TEXT,
                                edinichnaja_opuhol_-_storona_porazhenija TEXT,
                                diagnoz_-_morfologicheskij_tip_opuholi TEXT,
                                edinichnaja_opuhol_-_ostojatelstva_vyjavlenija_opuholi TEXT,
                                edinichnaja_opuhol_-_stadija_tnm TEXT,
                                edinichnaja_opuhol_-_klassifikatsija_ostatochnoj_opuholi_(r) TEXT,
                                edinichnaja_opuhol_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g) TEXT,
                                edinichnaja_opuhol_-_invazija_v_limfaticheskie_sosudy_(l) TEXT,
                                edinichnaja_opuhol_-_venoznaja_invazija_(v) TEXT,
                                edinichnaja_opuhol_-_perinevralnaja_invazija_(pn) TEXT,
                                edinichnaja_opuhol_-_lokalizatsija_otdalennyh_metastazov TEXT,
                                edinichnaja_opuhol_-_klinicheskaja_gruppa TEXT,
                                edinichnaja_opuhol_-_stadija_opuholevogo_protsessa TEXT,
                                biopsija_-_trebuetsja_vzjatie_biopsii? TEXT,
                                biopsija_-_prichina TEXT,_po_kotoroj_vzjatie_biopsii_ne_trebuetsja TEXT,
                                biopsija_-_data_provedenija_biopsii TEXT,
                                biopsija_-_otsutstvie_informatsii_o_date_provedenija_biopsii TEXT,
                                biopsija_-_gde_provedena_biopsija TEXT,
                                biopsija_-_gde_budet_provoditsja_biopsija_(tip_mo) TEXT,
                                biopsija_-_gde_budet_provoditsja_biopsija_(mo) TEXT,
                                biopsija_-_otsutstvie_informatsii_gde_provedena_biopsija TEXT,
                                onkokonsilium_-_trebuetsja_provedenie_onkokonsiliuma?_(true) TEXT,
                                onkokonsilium_-_planovaja_data_provedenija_onkokonsiliuma TEXT,
                                onkokonsilium_-_mesto_provedenija_onkokonsiliuma TEXT,
                                spetsializirovannoe_lechenie_-_trebuetsja_napravlenie_na_spetsializirovannoe_lechenie TEXT,
                                spetsializirovannoe_lechenie_-_hirurgicheskoe_lechenie TEXT,
                                spetsializirovannoe_lechenie_-_mesto_provedenija_hirurgicheskogo_lechenija TEXT,
                                spetsializirovannoe_lechenie_-_luchevoe_lechenie TEXT,
                                spetsializirovannoe_lechenie_-_lekarstvennoe_lechenie TEXT,
                                spetsializirovannoe_lechenie_-_gde_budet_provoditsja_lekarstvennoe_lechenie? TEXT,
                                spetsializirovannoe_lechenie_-_mesto_provedenija_lechenija TEXT,
                                spetsializirovannoe_lechenie_-_planovaja_data_provedenija_lekarstvennogo_lechenija TEXT,
                                spetsializirovannoe_lechenie_-_trebuetsja_naznachenie_lekarstvennoj_terapii? TEXT,
                                spetsializirovannoe_lechenie_-_mnn/tn_preparata TEXT,
                                spetsializirovannoe_lechenie_-_dozirovka TEXT,
                                dokumenty_na_prieme_spravki_i_napravlenija_-_tip_dokumenta TEXT,
                                dokumenty_na_prieme_spravki_i_napravlenija_-_instrumentalnye_obsledovanija TEXT,
                                dokumenty_na_prieme_spravki_i_napravlenija_-_laboratornye_issledovanija TEXT,
                                dokumenty_na_prieme_spravki_i_napravlenija_-_konsultatsija_spetsialista TEXT,
                                dokumenty_na_prieme_spravki_i_napravlenija_-_gospitalizatsija TEXT,
                                dokumenty_na_prieme_spravki_i_napravlenija_-_tsel_vydachi TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_kod_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_metod_podtverzhdenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_data_ustanovlenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_klinicheskaja_gruppa TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(1)_-_status_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_kod_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_metod_podtverzhdenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_data_ustanovlenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_klinicheskaja_gruppa TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(2)_-_status_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_kod_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_metod_podtverzhdenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_data_ustanovlenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_klinicheskaja_gruppa TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(3)_-_status_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_kod_mkb-10 TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_metod_podtverzhdenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_data_ustanovlenija_diagnoza TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_klinicheskaja_gruppa TEXT,
                                vlozhennyj_diagnoz_na_pervichnom_prieme_(4)_-_status_diagnoza TEXT,
                                edinichnaja_opuhol_-_ct TEXT,
                                edinichnaja_opuhol_-_cn TEXT,
                                edinichnaja_opuhol_-_cm TEXT,
                                edinichnaja_opuhol_-_pt TEXT,
                                edinichnaja_opuhol_-_pn TEXT,
                                edinichnaja_opuhol_-_pm TEXT,
                                mnozhestvennaja_opuhol(1)_-_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(1)_-_kod_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(1)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                mnozhestvennaja_opuhol(1)_-_status_diagnoza TEXT,
                                mnozhestvennaja_opuhol(1)_-_data_ustanovlenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(1)_-_osnovanie_diagnoza TEXT,
                                mnozhestvennaja_opuhol(1)_-_metod_podtverzhdenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(1)_-_priznak_osnovnoj_opuholi TEXT,
                                mnozhestvennaja_opuhol(1)_-_storona_porazhenija TEXT,
                                mnozhestvennaja_opuhol(1)_-_morfologicheskij_tip_opuholi TEXT,
                                mnozhestvennaja_opuhol(1)_-_obstojatelstva_vyjavlenija_opuholi TEXT,
                                mnozhestvennaja_opuhol(1)_-_stadija_tnm TEXT,
                                mnozhestvennaja_opuhol(1)_-_pt TEXT,
                                mnozhestvennaja_opuhol(1)_-_pn TEXT,
                                mnozhestvennaja_opuhol(1)_-_pm TEXT,
                                mnozhestvennaja_opuhol(1)_-_klassifikatsija_ostatochnoj_opuholi_(r) TEXT,
                                mnozhestvennaja_opuhol(1)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g) TEXT,
                                mnozhestvennaja_opuhol(1)_-_perinevralnaja_invazija_(pn) TEXT,
                                mnozhestvennaja_opuhol(1)_-_venoznaja_invazija_(v) TEXT,
                                mnozhestvennaja_opuhol(1)_-_invazija_v_limfaticheskie_sosudy_(l) TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.1 TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.2 TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.3 TEXT,
                                mnozhestvennaja_opuhol(1)_-_ct TEXT,
                                mnozhestvennaja_opuhol(1)_-_cn TEXT,
                                mnozhestvennaja_opuhol(1)_-_cm TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.4 TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.5 TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.6 TEXT,
                                mnozhestvennaja_opuhol(1)_-_lokalizatsija_otdalennyh_metastazov.7 TEXT,
                                mnozhestvennaja_opuhol(1)_-_stadija_opuholevogo_protsessa TEXT,
                                mnozhestvennaja_opuhol(1)_-_klinicheskaja_gruppa TEXT,
                                mnozhestvennaja_opuhol(2)_-_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(2)_-_kod_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(2)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                mnozhestvennaja_opuhol(2)_-_status_diagnoza TEXT,
                                mnozhestvennaja_opuhol(2)_-_data_ustanovlenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(2)_-_osnovanie_diagnoza TEXT,
                                mnozhestvennaja_opuhol(2)_-_metod_podtverzhdenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(2)_-_priznak_osnovnoj_opuholi TEXT,
                                mnozhestvennaja_opuhol(2)_-_storona_porazhenija TEXT,
                                mnozhestvennaja_opuhol(2)_-_morfologicheskij_tip_opuholi TEXT,
                                mnozhestvennaja_opuhol(2)_-_obstojatelstva_vyjavlenija_opuholi TEXT,
                                mnozhestvennaja_opuhol(2)_-_stadija_tnm TEXT,
                                mnozhestvennaja_opuhol(2)_-_pt TEXT,
                                mnozhestvennaja_opuhol(2)_-_pn TEXT,
                                mnozhestvennaja_opuhol(2)_-_pm TEXT,
                                mnozhestvennaja_opuhol(2)_-_klassifikatsija_ostatochnoj_opuholi_(r) TEXT,
                                mnozhestvennaja_opuhol(2)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g) TEXT,
                                mnozhestvennaja_opuhol(2)_-_perinevralnaja_invazija_(pn) TEXT,
                                mnozhestvennaja_opuhol(2)_-_venoznaja_invazija_(v) TEXT,
                                mnozhestvennaja_opuhol(2)_-_invazija_v_limfaticheskie_sosudy_(l) TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.8 TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.9 TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.10 TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.11 TEXT,
                                mnozhestvennaja_opuhol(2)_-_ct TEXT,
                                mnozhestvennaja_opuhol(2)_-_cn TEXT,
                                mnozhestvennaja_opuhol(2)_-_cm TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.12 TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.13 TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.14 TEXT,
                                mnozhestvennaja_opuhol(2)_-_lokalizatsija_otdalennyh_metastazov.15 TEXT,
                                mnozhestvennaja_opuhol(2)_-_stadija_opuholevogo_protsessa TEXT,
                                mnozhestvennaja_opuhol(2)_-_klinicheskaja_gruppa TEXT,
                                mnozhestvennaja_opuhol(3)_-_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(3)_-_kod_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(3)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                mnozhestvennaja_opuhol(3)_-_status_diagnoza TEXT,
                                mnozhestvennaja_opuhol(3)_-_data_ustanovlenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(3)_-_osnovanie_diagnoza TEXT,
                                mnozhestvennaja_opuhol(3)_-_metod_podtverzhdenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(3)_-_priznak_osnovnoj_opuholi TEXT,
                                mnozhestvennaja_opuhol(3)_-_storona_porazhenija TEXT,
                                mnozhestvennaja_opuhol(3)_-_morfologicheskij_tip_opuholi TEXT,
                                mnozhestvennaja_opuhol(3)_-_obstojatelstva_vyjavlenija_opuholi TEXT,
                                mnozhestvennaja_opuhol(3)_-_stadija_tnm TEXT,
                                mnozhestvennaja_opuhol(3)_-_pt TEXT,
                                mnozhestvennaja_opuhol(3)_-_pn TEXT,
                                mnozhestvennaja_opuhol(3)_-_pm TEXT,
                                mnozhestvennaja_opuhol(3)_-_klassifikatsija_ostatochnoj_opuholi_(r) TEXT,
                                mnozhestvennaja_opuhol(3)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g) TEXT,
                                mnozhestvennaja_opuhol(3)_-_perinevralnaja_invazija_(pn) TEXT,
                                mnozhestvennaja_opuhol(3)_-_venoznaja_invazija_(v) TEXT,
                                mnozhestvennaja_opuhol(3)_-_invazija_v_limfaticheskie_sosudy_(l) TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.16 TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.17 TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.18 TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.19 TEXT,
                                mnozhestvennaja_opuhol(3)_-_ct TEXT,
                                mnozhestvennaja_opuhol(3)_-_cn TEXT,
                                mnozhestvennaja_opuhol(3)_-_cm TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.20 TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.21 TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.22 TEXT,
                                mnozhestvennaja_opuhol(3)_-_lokalizatsija_otdalennyh_metastazov.23 TEXT,
                                mnozhestvennaja_opuhol(3)_-_stadija_opuholevogo_protsessa TEXT,
                                mnozhestvennaja_opuhol(3)_-_klinicheskaja_gruppa TEXT,
                                mnozhestvennaja_opuhol(4)_-_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(4)_-_kod_mkb-10 TEXT,
                                mnozhestvennaja_opuhol(4)_-_razvernutyj_klinicheskij_diagnoz TEXT,
                                mnozhestvennaja_opuhol(4)_-_status_diagnoza TEXT,
                                mnozhestvennaja_opuhol(4)_-_data_ustanovlenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(4)_-_osnovanie_diagnoza TEXT,
                                mnozhestvennaja_opuhol(4)_-_metod_podtverzhdenija_diagnoza TEXT,
                                mnozhestvennaja_opuhol(4)_-_priznak_osnovnoj_opuholi TEXT,
                                mnozhestvennaja_opuhol(4)_-_storona_porazhenija TEXT,
                                mnozhestvennaja_opuhol(4)_-_morfologicheskij_tip_opuholi TEXT,
                                mnozhestvennaja_opuhol(4)_-_obstojatelstva_vyjavlenija_opuholi TEXT,
                                mnozhestvennaja_opuhol(4)_-_stadija_tnm TEXT,
                                mnozhestvennaja_opuhol(4)_-_pt TEXT,
                                mnozhestvennaja_opuhol(4)_-_pn TEXT,
                                mnozhestvennaja_opuhol(4)_-_pm TEXT,
                                mnozhestvennaja_opuhol(4)_-_klassifikatsija_ostatochnoj_opuholi_(r) TEXT,
                                mnozhestvennaja_opuhol(4)_-_gistopatologicheskaja_stepen_zlokachestvennosti_(g) TEXT,
                                mnozhestvennaja_opuhol(4)_-_perinevralnaja_invazija_(pn) TEXT,
                                mnozhestvennaja_opuhol(4)_-_venoznaja_invazija_(v) TEXT,
                                mnozhestvennaja_opuhol(4)_-_invazija_v_limfaticheskie_sosudy_(l) TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.24 TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.25 TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.26 TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.27 TEXT,
                                mnozhestvennaja_opuhol(4)_-_ct TEXT,
                                mnozhestvennaja_opuhol(4)_-_cn TEXT,
                                mnozhestvennaja_opuhol(4)_-_cm TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.28 TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.29 TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.30 TEXT,
                                mnozhestvennaja_opuhol(4)_-_lokalizatsija_otdalennyh_metastazov.31 TEXT,
                                mnozhestvennaja_opuhol(4)_-_stadija_opuholevogo_protsessa TEXT,
                                mnozhestvennaja_opuhol(4)_-_klinicheskaja_gruppa TEXT,
                                tsaop TEXT,
                                tsaop_id TEXT,
                                tsaop_oms TEXT,
                                tsaop_v ТЕХТ);'''

    cursor = sqlite_connection.cursor()
    print("База данных подключена к SQLite")
    
    cursor.execute(sqlite_create_table_query)
    sqlite_connection.commit()
    print("Таблица SQLite создана")

    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")

База данных подключена к SQLite
Ошибка при подключении к sqlite near "-": syntax error
Соединение с SQLite закрыто


In [30]:
sqlite_connection = sqlite3.connect('test_sqlite_python.db')
sqlite_create_table_query = '''CREATE TABLE test_sqlite_db (
                    inner_id INTEGER PRIMARY KEY,
                    tip_priema TEXT,
                    obschaja_informatsija__id_protokola TEXT,
                    bschaja_informatsija__id_patsienta TEXT,
                    obschaja_informatsija__data_i_vremja_podpisanija_protokola ТЕХТ,
                    obschaja_informatsija__fiopatsienta TEXT,
                    obschaja_informatsija__dr_patsienta ТЕХТ,
                    obschaja_informatsija__oms_patsienta TEXT,
                    obschaja_informatsija__fio_vracha TEXT,
                    obschaja_informatsija__dolzhnost_vracha TEXT,
                    obschaja_informatsija__mo_priema TEXT,
                    obschaja_informatsija__strukturnoe_podrazdelenie TEXT,
                    obschaja_informatsija__filial TEXT,
                    obschaja_informatsija__otdelenie TEXT,
                    obschaja_informatsija__spetsializatsija_vracha TEXT,
                    prikreplenie__id_uchastka_prikreplenija_v_mu_priema TEXT,
                    prikreplenie__uchastok_prikreplenija_v_mu_priema TEXT,
                    prikreplenie__prikreplenija_v_drugih_mo TEXT,
                    anamnez TEXT,
                    zhaloby_predjavljaet___ne_predjavljaet TEXT,
                    zhaloby TEXT,
                    opisanie_zhalob TEXT,
                    schitaet_sebja_bolnym_s_ TEXT,
                    obschee_sostojanie TEXT,
                    uroven_soznanija TEXT,
                    nervnopsihicheskij_status TEXT,
                    nps_inoe TEXT,
                    sostojanie_patsienta_ecog TEXT,
                    temperatura_tela TEXT,
                    lokalnyj_status TEXT,
                    napravitelnyj_diagnoz__mkb10 TEXT,
                    napravitelnyj_diagnoz__kod_mkb10 TEXT,
                    diagnoz__mkb10_rasshifrovka TEXT,
                    diagnoz__kod_mkb10 TEXT,
                    diagnoz__razvernutyj_klinicheskij_diagnoz TEXT,
                    diagnoz__data_ustanvoki_diagnoza TEXT,
                    diagnoz__status_diagnoza TEXT,
                    diagnoz__pervichnomnozhestvennaja_opuhol_da___net TEXT,
                    diagnoz__metod_podtverzhdenija_diagnoza TEXT,
                    diagnoz__klinicheskaja_gruppa TEXT,
                    diagnoz__osnovanie_diagnoza TEXT,
                    prochie_rekomendatsii TEXT,
                    novyj_diagnoz__mkb10 TEXT,
                    novyj_diagnoz__kod_mkb10 TEXT,
                    novyj_diagnoz__data_ustanovlenija_novogo_diagnoza TEXT,
                    novyj_diagnoz__status_novogo_diagnoza TEXT,
                    edinichnaja_opuhol__storona_porazhenija TEXT,
                    diagnoz__morfologicheskij_tip_opuholi TEXT,
                    edinichnaja_opuhol__ostojatelstva_vyjavlenija_opuholi TEXT,
                    edinichnaja_opuhol__stadija_tnm TEXT,
                    edinichnaja_opuhol__klassifikatsija_ostatochnoj_opuholi_r TEXT,
                    edinichnaja_opuhol__gistopatologicheskaja_stepen_zlokachestvennosti_g TEXT,
                    edinichnaja_opuhol__invazija_v_limfaticheskie_sosudy_l TEXT,
                    edinichnaja_opuhol__venoznaja_invazija_v TEXT,
                    edinichnaja_opuhol__perinevralnaja_invazija_pn TEXT,
                    edinichnaja_opuhol__lokalizatsija_otdalennyh_metastazov TEXT,
                    edinichnaja_opuhol__klinicheskaja_gruppa TEXT,
                    edinichnaja_opuhol__stadija_opuholevogo_protsessa TEXT,
                    biopsija__trebuetsja_vzjatie_biopsii___ TEXT,
                    biopsija__prichina TEXT,_po_kotoroj_vzjatie_biopsii_ne_trebuetsja TEXT,
                    biopsija__data_provedenija_biopsii TEXT,
                    biopsija__otsutstvie_informatsii_o_date_provedenija_biopsii TEXT,
                    biopsija__gde_provedena_biopsija TEXT,
                    biopsija__gde_budet_provoditsja_biopsija_tip_mo TEXT,
                    biopsija__gde_budet_provoditsja_biopsija_mo TEXT,
                    biopsija__otsutstvie_informatsii_gde_provedena_biopsija TEXT,
                    onkokonsilium__trebuetsja_provedenie_onkokonsiliuma____true TEXT,
                    onkokonsilium__planovaja_data_provedenija_onkokonsiliuma TEXT,
                    onkokonsilium__mesto_provedenija_onkokonsiliuma TEXT,
                    spetsializirovannoe_lechenie__trebuetsja_napravlenie_na_spetsializirovannoe_lechenie TEXT,
                    spetsializirovannoe_lechenie__hirurgicheskoe_lechenie TEXT,
                    spetsializirovannoe_lechenie__mesto_provedenija_hirurgicheskogo_lechenija TEXT,
                    spetsializirovannoe_lechenie__luchevoe_lechenie TEXT,
                    spetsializirovannoe_lechenie__lekarstvennoe_lechenie TEXT,
                    spetsializirovannoe_lechenie__gde_budet_provoditsja_lekarstvennoe_lechenie___ TEXT,
                    spetsializirovannoe_lechenie__mesto_provedenija_lechenija TEXT,
                    spetsializirovannoe_lechenie__planovaja_data_provedenija_lekarstvennogo_lechenija TEXT,
                    spetsializirovannoe_lechenie__trebuetsja_naznachenie_lekarstvennoj_terapii___ TEXT,
                    spetsializirovannoe_lechenie__mnn___tn_preparata TEXT,
                    spetsializirovannoe_lechenie__dozirovka TEXT,
                    dokumenty_na_prieme_spravki_i_napravlenija__tip_dokumenta TEXT,
                    dokumenty_na_prieme_spravki_i_napravlenija__instrumentalnye_obsledovanija TEXT,
                    dokumenty_na_prieme_spravki_i_napravlenija__laboratornye_issledovanija TEXT,
                    dokumenty_na_prieme_spravki_i_napravlenija__konsultatsija_spetsialista TEXT,
                    dokumenty_na_prieme_spravki_i_napravlenija__gospitalizatsija TEXT,
                    dokumenty_na_prieme_spravki_i_napravlenija__tsel_vydachi TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__kod_mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__razvernutyj_klinicheskij_diagnoz TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__metod_podtverzhdenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__data_ustanovlenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__klinicheskaja_gruppa TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_1__status_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__kod_mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__razvernutyj_klinicheskij_diagnoz TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__metod_podtverzhdenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__data_ustanovlenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__klinicheskaja_gruppa TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_2__status_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__kod_mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__razvernutyj_klinicheskij_diagnoz TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__metod_podtverzhdenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__data_ustanovlenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__klinicheskaja_gruppa TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_3__status_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__kod_mkb10 TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__razvernutyj_klinicheskij_diagnoz TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__metod_podtverzhdenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__data_ustanovlenija_diagnoza TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__klinicheskaja_gruppa TEXT,
                    vlozhennyj_diagnoz_na_pervichnom_prieme_4__status_diagnoza TEXT,
                    edinichnaja_opuhol__ct TEXT,
                    edinichnaja_opuhol__cn TEXT,
                    edinichnaja_opuhol__cm TEXT,
                    edinichnaja_opuhol__pt TEXT,
                    edinichnaja_opuhol__pn TEXT,
                    edinichnaja_opuhol__pm TEXT,
                    mnozhestvennaja_opuhol1__mkb10 TEXT,
                    mnozhestvennaja_opuhol1__kod_mkb10 TEXT,
                    mnozhestvennaja_opuhol1__razvernutyj_klinicheskij_diagnoz TEXT,
                    mnozhestvennaja_opuhol1__status_diagnoza TEXT,
                    mnozhestvennaja_opuhol1__data_ustanovlenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol1__osnovanie_diagnoza TEXT,
                    mnozhestvennaja_opuhol1__metod_podtverzhdenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol1__priznak_osnovnoj_opuholi TEXT,
                    mnozhestvennaja_opuhol1__storona_porazhenija TEXT,
                    mnozhestvennaja_opuhol1__morfologicheskij_tip_opuholi TEXT,
                    mnozhestvennaja_opuhol1__obstojatelstva_vyjavlenija_opuholi TEXT,
                    mnozhestvennaja_opuhol1__stadija_tnm TEXT,
                    mnozhestvennaja_opuhol1__pt TEXT,
                    mnozhestvennaja_opuhol1__pn TEXT,
                    mnozhestvennaja_opuhol1__pm TEXT,
                    mnozhestvennaja_opuhol1__klassifikatsija_ostatochnoj_opuholi_r TEXT,
                    mnozhestvennaja_opuhol1__gistopatologicheskaja_stepen_zlokachestvennosti_g TEXT,
                    mnozhestvennaja_opuhol1__perinevralnaja_invazija_pn TEXT,
                    mnozhestvennaja_opuhol1__venoznaja_invazija_v TEXT,
                    mnozhestvennaja_opuhol1__invazija_v_limfaticheskie_sosudy_l TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_1 TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_2 TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_3 TEXT,
                    mnozhestvennaja_opuhol1__ct TEXT,
                    mnozhestvennaja_opuhol1__cn TEXT,
                    mnozhestvennaja_opuhol1__cm TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_4 TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_5 TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_6 TEXT,
                    mnozhestvennaja_opuhol1__lokalizatsija_otdalennyh_metastazov_7 TEXT,
                    mnozhestvennaja_opuhol1__stadija_opuholevogo_protsessa TEXT,
                    mnozhestvennaja_opuhol1__klinicheskaja_gruppa TEXT,
                    mnozhestvennaja_opuhol2__mkb10 TEXT,
                    mnozhestvennaja_opuhol2__kod_mkb10 TEXT,
                    mnozhestvennaja_opuhol2__razvernutyj_klinicheskij_diagnoz TEXT,
                    mnozhestvennaja_opuhol2__status_diagnoza TEXT,
                    mnozhestvennaja_opuhol2__data_ustanovlenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol2__osnovanie_diagnoza TEXT,
                    mnozhestvennaja_opuhol2__metod_podtverzhdenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol2__priznak_osnovnoj_opuholi TEXT,
                    mnozhestvennaja_opuhol2__storona_porazhenija TEXT,
                    mnozhestvennaja_opuhol2__morfologicheskij_tip_opuholi TEXT,
                    mnozhestvennaja_opuhol2__obstojatelstva_vyjavlenija_opuholi TEXT,
                    mnozhestvennaja_opuhol2__stadija_tnm TEXT,
                    mnozhestvennaja_opuhol2__pt TEXT,
                    mnozhestvennaja_opuhol2__pn TEXT,
                    mnozhestvennaja_opuhol2__pm TEXT,
                    mnozhestvennaja_opuhol2__klassifikatsija_ostatochnoj_opuholi_r TEXT,
                    mnozhestvennaja_opuhol2__gistopatologicheskaja_stepen_zlokachestvennosti_g TEXT,
                    mnozhestvennaja_opuhol2__perinevralnaja_invazija_pn TEXT,
                    mnozhestvennaja_opuhol2__venoznaja_invazija_v TEXT,
                    mnozhestvennaja_opuhol2__invazija_v_limfaticheskie_sosudy_l TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_8 TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_9 TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_10 TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_11 TEXT,
                    mnozhestvennaja_opuhol2__ct TEXT,
                    mnozhestvennaja_opuhol2__cn TEXT,
                    mnozhestvennaja_opuhol2__cm TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_12 TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_13 TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_14 TEXT,
                    mnozhestvennaja_opuhol2__lokalizatsija_otdalennyh_metastazov_15 TEXT,
                    mnozhestvennaja_opuhol2__stadija_opuholevogo_protsessa TEXT,
                    mnozhestvennaja_opuhol2__klinicheskaja_gruppa TEXT,
                    mnozhestvennaja_opuhol3__mkb10 TEXT,
                    mnozhestvennaja_opuhol3__kod_mkb10 TEXT,
                    mnozhestvennaja_opuhol3__razvernutyj_klinicheskij_diagnoz TEXT,
                    mnozhestvennaja_opuhol3__status_diagnoza TEXT,
                    mnozhestvennaja_opuhol3__data_ustanovlenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol3__osnovanie_diagnoza TEXT,
                    mnozhestvennaja_opuhol3__metod_podtverzhdenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol3__priznak_osnovnoj_opuholi TEXT,
                    mnozhestvennaja_opuhol3__storona_porazhenija TEXT,
                    mnozhestvennaja_opuhol3__morfologicheskij_tip_opuholi TEXT,
                    mnozhestvennaja_opuhol3__obstojatelstva_vyjavlenija_opuholi TEXT,
                    mnozhestvennaja_opuhol3__stadija_tnm TEXT,
                    mnozhestvennaja_opuhol3__pt TEXT,
                    mnozhestvennaja_opuhol3__pn TEXT,
                    mnozhestvennaja_opuhol3__pm TEXT,
                    mnozhestvennaja_opuhol3__klassifikatsija_ostatochnoj_opuholi_r TEXT,
                    mnozhestvennaja_opuhol3__gistopatologicheskaja_stepen_zlokachestvennosti_g TEXT,
                    mnozhestvennaja_opuhol3__perinevralnaja_invazija_pn TEXT,
                    mnozhestvennaja_opuhol3__venoznaja_invazija_v TEXT,
                    mnozhestvennaja_opuhol3__invazija_v_limfaticheskie_sosudy_l TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_16 TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_17 TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_18 TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_19 TEXT,
                    mnozhestvennaja_opuhol3__ct TEXT,
                    mnozhestvennaja_opuhol3__cn TEXT,
                    mnozhestvennaja_opuhol3__cm TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_20 TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_21 TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_22 TEXT,
                    mnozhestvennaja_opuhol3__lokalizatsija_otdalennyh_metastazov_23 TEXT,
                    mnozhestvennaja_opuhol3__stadija_opuholevogo_protsessa TEXT,
                    mnozhestvennaja_opuhol3__klinicheskaja_gruppa TEXT,
                    mnozhestvennaja_opuhol4__mkb10 TEXT,
                    mnozhestvennaja_opuhol4__kod_mkb10 TEXT,
                    mnozhestvennaja_opuhol4__razvernutyj_klinicheskij_diagnoz TEXT,
                    mnozhestvennaja_opuhol4__status_diagnoza TEXT,
                    mnozhestvennaja_opuhol4__data_ustanovlenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol4__osnovanie_diagnoza TEXT,
                    mnozhestvennaja_opuhol4__metod_podtverzhdenija_diagnoza TEXT,
                    mnozhestvennaja_opuhol4__priznak_osnovnoj_opuholi TEXT,
                    mnozhestvennaja_opuhol4__storona_porazhenija TEXT,
                    mnozhestvennaja_opuhol4__morfologicheskij_tip_opuholi TEXT,
                    mnozhestvennaja_opuhol4__obstojatelstva_vyjavlenija_opuholi TEXT,
                    mnozhestvennaja_opuhol4__stadija_tnm TEXT,
                    mnozhestvennaja_opuhol4__pt TEXT,
                    mnozhestvennaja_opuhol4__pn TEXT,
                    mnozhestvennaja_opuhol4__pm TEXT,
                    mnozhestvennaja_opuhol4__klassifikatsija_ostatochnoj_opuholi_r TEXT,
                    mnozhestvennaja_opuhol4__gistopatologicheskaja_stepen_zlokachestvennosti_g TEXT,
                    mnozhestvennaja_opuhol4__perinevralnaja_invazija_pn TEXT,
                    mnozhestvennaja_opuhol4__venoznaja_invazija_v TEXT,
                    mnozhestvennaja_opuhol4__invazija_v_limfaticheskie_sosudy_l TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_24 TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_25 TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_26 TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_27 TEXT,
                    mnozhestvennaja_opuhol4__ct TEXT,
                    mnozhestvennaja_opuhol4__cn TEXT,
                    mnozhestvennaja_opuhol4__cm TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_28 TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_29 TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_30 TEXT,
                    mnozhestvennaja_opuhol4__lokalizatsija_otdalennyh_metastazov_31 TEXT,
                    mnozhestvennaja_opuhol4__stadija_opuholevogo_protsessa TEXT,
                    mnozhestvennaja_opuhol4__klinicheskaja_gruppa TEXT,
                    tsaop TEXT,
                    tsaop_id TEXT,
                    tsaop_oms TEXT,
                    tsaop_v ТЕХТ);'''

In [31]:
cursor = sqlite_connection.cursor()
cursor.execute(sqlite_create_table_query)
sqlite_connection.commit()

In [ ]:
try:
    sqlite_connection = sqlite3.connect('sqlite_python.db')
    sqlite_create_table_query = '''CREATE TABLE sqlitedb_developers (
                                id INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                email text NOT NULL UNIQUE,
                                joining_date datetime,
                                salary REAL NOT NULL);'''

    cursor = sqlite_connection.cursor()
    print("База данных подключена к SQLite")
    cursor.execute(sqlite_create_table_query)
    sqlite_connection.commit()